<div class="alert alert-info">
    
➡️ Before you start, make sure that you are familiar with the **[study guide](https://liu-nlp.ai/text-mining/logistics/)**, in particular the rules around **cheating and plagiarism** (found in the course memo).

➡️ If you use code from external sources (e.g. StackOverflow, ChatGPT, ...) as part of your solutions, don't forget to add a reference to these source(s) (for example as a comment above your code).

➡️ Make sure you fill in all cells that say **`YOUR CODE HERE`** or **YOUR ANSWER HERE**.  You normally shouldn't need to modify any of the other cells.

</div>

# L1: Information Retrieval

In this lab you will apply basic techniques from information retrieval to implement the core of a minimalistic search engine. The data for this lab consists of a collection of app descriptions scraped from the [Google Play Store](https://play.google.com/store/apps?hl=en). From this collection, your search engine should retrieve those apps whose descriptions best match a given query under the vector space model.

In [1]:
# Define some helper functions that are used in this notebook
from IPython.display import display, HTML

def success():
    display(HTML('<div class="alert alert-success"><strong>Checks have passed!</strong></div>'))

## Dataset

The app descriptions come in the form of a compressed [JSON](https://en.wikipedia.org/wiki/JSON) file. Start by loading this file into a [Pandas](https://pandas.pydata.org) [DataFrame](https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html#dataframe).

In [2]:
import bz2
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', 500)

with bz2.open('app-descriptions.json.bz2', mode='rt', encoding='utf-8') as source:
    df = pd.read_json(source, encoding='utf-8')

In Pandas, a DataFrame is a table with indexed rows and labelled columns of potentially different types. You can access data in a DataFrame in various ways, including by row and column. To give an example, the code in the next cell shows rows 200–204:

In [3]:
df.loc[200:205]

,name,description
200,Brick Breaker Star: Space King,"Introducing the best Brick Breaker game that everyone can enjoy.\nEnjoy various missions and addictively simple play control.\n\n[Features]\n- Hundreds of stages and various missions\n- No limit to play such as Heart, play as much as you can!\n- 5 kinds of various items and items reinforcement system\n- No network required\n- game file is as low as 20M, light-weight download!\n- supports tablet screen\n- supports Google Play Leaderboards, Achievement, Multiplay\n- supports 14 languages\n\nHo..."
201,Brick Classic - Brick Game,"Classic Brick Game!\n\nBrick Classic is a popular and addictive puzzle game!\n\nHow to play?\n- Simply drag the bricks to move them.\n- Create full lines on the grid vertically or horizontally to break bricks.\n\nTips:\n- Classic brick game without time limits.\n- Place the bricks in a reasonable position.\n- The more brick break, the more scores you have.\n- Bricks can't be rotated.\n\nWho's the best brick breaker? Challenge it now!!!"
202,Bricks Breaker - Glow Balls,"Bricks Breaker - Glow Balls is a addictive and challenging brick game.\nJust play it to relax your brain. Be focus on breaking bricks and you will find it more funny and exciting.\n\nHow to play\n- Hold the screen with your finger and move to aim.\n- Find best positions and angles to hit all bricks.\n- When the durability of brick reaches 0, destroyed.\n- Never let bricks reach the bottom or game is over.\n\nFeatures\n- Colorful glow skins.\n- Free to play.\n- Easy game controls with one fin..."
203,Bricks Breaker Quest,"How to play\n- The ball flies to wherever you touched.\n- Clear the stages by removing bricks on the board.\n- Break the bricks and never let them hit the bottom.\n- Find best positions and angles to hit every brick.\n\nFeature\n- Free to play\n- Tons of stages\n- Various types of balls\n- Easy to play, Simplest game system, Designed for one handheld gameplay.\n- Off-line (without internet connection) gameplay supported \n- Multi-play supported\n- Tablet device supported\n- Achievement & lea..."
204,Brothers in Arms® 3,"Fight brave soldiers from around the globe on the frenzied multiplayer battlegrounds of World War 2 or become Sergeant Wright and experience a dramatic, life-changing single-player journey, in the aftermath of the D-Day invasion.\n\nCLIMB THE ARMY RANKS IN MULTIPLAYER \n> 4 maps to master and enjoy. \n> 2 gameplay modes to begin with: Free For All and Team Deathmatch.\n> Unlock game-changing perks by playing with each weapon class!\n> A soldier’s only as deadly as his weapon. Be sure to upgr..."
205,Brown Dust - Tactical RPG,"The Empire has fallen, and the Age of Great Mercenaries Now Begins!\nCreate Your Ultimate Team And Strike Down Your Enemies!\n\nCAPTIVATING AND STUNNING ARTWORK\n- Experience the high-quality anime illustrations you have never seen before.\n- Meet Brown Dust's charming Mercenaries now.\n\nASSEMBLE LEGENDARY MERCENARIES\n- Over 300 Mercenaries and a Variety of Skills.\n- Discover the Unique Mercenaries, 6 Devils and Dominus Octo.\n- All Mercenaries can reach max level and the highest rank.\n\..."


As you can see, there are two labelled columns: `name` (the name of the app) and `description` (a textual description). The next cell shows how to access only the description field from row 200:

In [4]:
df.loc[200, 'description']

'Introducing the best Brick Breaker game that everyone can enjoy.\nEnjoy various missions and addictively simple play control.\n\n[Features]\n- Hundreds of stages and various missions\n- No limit to play such as Heart, play as much as you can!\n- 5 kinds of various items and items reinforcement system\n- No network required\n- game file is as low as 20M, light-weight download!\n- supports tablet screen\n- supports Google Play Leaderboards, Achievement, Multiplay\n- supports 14 languages\n\nHomepage:\nhttps://play.google.com/store/apps/dev?id=4931745640662708567\n\nFacebook: \nhttps://www.facebook.com/spcomesgames/'

## Problem 1: What's in a vector?

We start by vectorising the data — more specifically, we map each app description to a tf–idf vector. This is very simple with a library like [scikit-learn](https://scikit-learn.org/stable/), which provides a [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) class for exactly this purpose.  If we instantiate this class, and call `fit_transform()` on all of our app descriptions, scikit-learn will preprocess and tokenize each app description, compute tf–idf values for each of them, and return a vectorised representation:

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['description'])
X

<1614x27877 sparse matrix of type '<class 'numpy.float64'>'
	with 267110 stored elements in Compressed Sparse Row format>

Let’s pick the app "Pancake Tower", which has a rather short description text, to see how it has been vectorised:

In [6]:
# We can use 'toarray' to convert the sparse matrix object into a "normal" array
vec = X[1032].toarray()[0]

# The app description & its corresponding vector
df.loc[1032, 'description'], vec

("Let's see how many pancakes you can pile up!!",
 array([0., 0., 0., ..., 0., 0., 0.]))

That's not very informative yet.  We know that the vector contains tf–idf values, and that each dimension of the vector corresponds to a token in the vectorizer’s vocabulary; let's extract these for this specific example.

Your **first task** is to find out how to access the `vectorizer`’s vocabulary, for example by [checking the documentation of `TfidfVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html), and print all the tokens that are represented in the vector with a tf–idf value greater than zero (i.e., only the tokens that are actually part of this app’s description) _in descending order of the tf–idf values_.  In other words, the token with the highest tf–idf value should be at the top of your output, and the token with the lowest tf–idf value at the bottom.   Before you implement this, think about what you would expect the output look like, for example which words you would expect to have the highest/lowest tf–idf values in this example.

Your final output should look something like this:

```
<token 1>: <tf-idf value 1>
<token 2>: <tf-idf value 2>
...
```

In [7]:
"""Print the tokens and their tf–idf values, in descending order."""

# YOUR CODE HERE
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer.get_feature_names_out
out_feature_names = vectorizer.get_feature_names_out()
# https://www.geeksforgeeks.org/zip-in-python/
token_tfidfval = [(token, tfidfval) for token, tfidfval in zip(out_feature_names,vec) if tfidfval>0]
#https://www.geeksforgeeks.org/sort-in-python/?ref=header_outind
token_tfidfval.sort(key=lambda x: x[1], reverse=True)
for token, tfidfval in token_tfidfval:
    print(f"{token}: {tfidfval}")
# raise NotImplementedError()

pancakes: 0.6539332651185913
pile: 0.5304701435508047
let: 0.2615287714771797
see: 0.2557630827415271
many: 0.23491959669849022
how: 0.21153246225085887
up: 0.17216837691451817
can: 0.13047602895910532
you: 0.10276923239718011


## Problem 2: Finding the nearest vectors

To build a small search engine, we need to be able to turn _queries_ (for example the string "pile up pancakes") into _query vectors_, and then find out which of our app description vectors are closest to the query vector.

For the first part (turning queries into query vectors), we can simply re-use the `vectorizer` that we used for the app descriptions. For the second part, an easy way to find the closest vectors is to use scikit-learn’s [NearestNeighbors](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html) class. This class needs to be _fit_ on a set of vectors (the "training set"; in our case the app descriptions) and can then be used with any vector to find its _nearest neighbors_ in the vector space.

**First,** instantiate and fit a class that returns the _ten (10)_ nearest neighbors:

In [8]:
"""Instantiate and fit a class that returns the 10 nearest neighboring vectors."""

# YOUR CODE HERE
from sklearn.neighbors import NearestNeighbors
# https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html
neighbors_nearest = NearestNeighbors(n_neighbors=10)
neighbors_nearest.fit(X)

# raise NotImplementedError()

NearestNeighbors(n_neighbors=10)

**Second,** implement a function that uses the vectorizer and the fitted class to find the nearest neighbours for a given query string:

In [9]:
def search(query):
    """Find the nearest neighbors in `df` for a query string.

    Arguments:
      query (str): A query string.

    Returns:
      The 10 apps (with name and description) most similar (in terms of
      cosine similarity) to the given query as a Pandas DataFrame.
    """
    # YOUR CODE HERE
    query_vector = vectorizer.transform([query])
    # https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html
    neigh_dist, neigh_ind = neighbors_nearest.kneighbors(query_vector)
    return(df.iloc[neigh_ind[0]][["name","description"]])
    raise NotImplementedError()

### 🤞 Test your code

Test your implementation by running the following cell, which will sanity-check your return value and show the 10 best search results for the query _"pile up pancakes"_:

In [10]:
"""Check that searching for "pile up pancakes" returns a DataFrame with ten results,
   and that the top result is "Pancake Tower"."""

result = search('pile up pancakes')
display(result)
assert isinstance(result, pd.DataFrame), "search() function should return a Pandas DataFrame"
assert len(result) == 10, "search() function should return 10 search results"
assert result.iloc[0]["name"] == "Pancake Tower", "Top search result should be 'Pancake Tower'"
success()

,name,description
1032,Pancake Tower,Let's see how many pancakes you can pile up!!
326,Cooking School: Games for Girls,"Children like to help their parents. They especially like to help with cooking . When there is a cooking in the kitchen, it is no way to play. But cooking is a complicated process and often it ends up with a huge mess in the kitchen. But what if you are so eager to cook pancakes, cake or cupcakes? How to cook all that without doing a cleaning after? We have a solution! Home Cooking School with our curious Hippo has opened especially for parents and children! We do not only cook food here. We..."
656,"Hell’s Cooking — crazy chef burger, kitchen fever","⭐ ⭐ ⭐ ⭐ ⭐ New world of crazy cooking is here. Feel what it means to be a master chef who prepares fantastic fast food in a prominent king kitchen! If you haven't ever tried yourself as a hamburger chef cook, it's possibly the best time for making diner. Download and launch Hell's Cooking — crazy chef burger, kitchen fever HD game and get prepared to jump into a fever and adventurous perfect world of burgers.\n\nNew girls game Hell's Cooking gives you lots of opportunities for your crazy cafe..."
1235,Solitaire,"Solitaire Free by Solitaire Card Games is the #1 klondike solitaire games on android. The solitaire Free is popular and classic card games you know and love.\n\nWe carefully designed a fresh solitaire free modern look, woven into the wonderful solitaire classic feel that everyone loves. \n\nExperience the crisp, clear, and easy to read cards, simple and quick animations, and subtle sounds, in either landscape or portrait views. \n\nYou can move cards with a single tap or drag them to their d..."
1164,Rummy - Free,"Play the famous Rummy card game on your Android Smartphone or Tablet !! \n\nPlay rummy with 2, 3, or 4 players against simulated opponents playing with high-level artificial intelligence. \nThere are a number of rules that can be modified, making this game very faithful to the original. \n\n*** MANY VARIATIONS INCLUDED *** \n\nMany rummy variations are included in the application: \n\n- From 2 to 4 players. \n- Choose the AI level of opponents. \n- Number of cards dealt to each player (from ..."
1181,Sago Mini Trucks and Diggers,"Drive a dump truck with Rosie the hamster! Pile dirt high and dig deep in the ground with diggers, cranes and bulldozers. Build a home for a new friend! Choose a barn, a castle or even a cupcake-house. Don’t forget to add the finishing touches for the proud owner.\n\nOn this construction site, kids love being the boss. With six mighty machines and piles of dirt, you can build all day! Part of the award-winning suite of Sago Mini apps, this app puts kids in charge.\n\nSago Mini apps have no i..."
436,Dr. Panda's Ice Cream Truck,"Chocolate? Vanilla? Strawberry? All three!? You decide! In Dr. Panda’s Ice Cream Truck you can mix up all sorts of different flavors with cookies, chocolate, nuts and more to make the perfect ice cream—hundreds of combinations in all.\n\nScoop it!\nThese animals love ice cream, and will eat as much (or little) as you want to serve them. You can make scoops big or small and pile them as high as you want—using any of the ice cream you’ve created!\n\nToppings galore!\nUse chocolate syrup, cooki..."
1442,Turbo Dismount™,"The legendary crash simulator is now on Google Play!\n\nPerform death-defying motor stunts, crash into walls, create traffic pile-ups of epic scale - and share the fun!\n\nTurbo Dismount™ is a kinetic tragedy about Mr. Dismount and the cars who love him. It is the official sequel to the wildly popular and immensely successful personal impact simulator - Stair Dismount™. \n\nFEATURES:\n* Flinch-inducing crash physics\n* Crunchy sound effects\n* Delicious slow-mo replay system\n* Multiple vehi..."
1446,UNO!™,"Play the world’s number one card game like never before. UNO!™ has all-new rules, tournaments, adventures and so much more! At home or on the move, jump into games instantly. Whet

Before continuing with the next problem, play around a bit with this simple search functionality by trying out different search queries, and see if the results look like what you would expect:

In [11]:
# Example — try out your own queries!
search("dodge trains")

,name,description
1428,Train Conductor World,"Master and manage the chaos of international railway traffic as the ultimate railroad tycoon. Build the rail network of your dreams; lay rails and solve the railroad puzzle with branching and forking roads at every turn. Become the richest manager and pick your path, do you optimise to the micro level, planning routes and managing the timetable, or sit idle letting your business keep earning while you sleep! \n\nGet in the driver's seat and take passengers to their destinations, dropping the..."
1301,Subway Surfers,"DASH as fast as you can! \nDODGE the oncoming trains! \n\nHelp Jake, Tricky & Fresh escape from the grumpy Inspector and his dog. \n\n★ Grind trains with your cool crew! \n★ Colorful and vivid HD graphics! \n★ Hoverboard Surfing! \n★ Paint powered jetpack! \n★ Lightning fast swipe acrobatics! \n★ Challenge and help your friends! \n\nJoin the most daring chase! \n\nA Universal App with HD optimized graphics.\n\nBy Kiloo and Sybo."
1300,Subway Princess Runner,"Subway princess runner, Bus run, forest rush with addictive endless running game!\nRush as fast as you can, dodge the oncoming trains and buses. Careful the rolling wood in the forest! Intuitive controls to run left or right, jump in the sky to obtain more coins, excited slide to safety!\n\nHelp your loved beautiful princess to escape the police! Use skateboard after double tapping, experience the unique board in the subway. Challenge the highest score of the rank with the world players or s..."
998,No Humanity - The Hardest Game,"2M+ Downloads All Over The World!\n\n* IGN Nominated Best Aussie/NZ game *\n* Top 5 indie games at PAX 2015 Australia – Mashable *\n* Global Game Jam ""Best Game"" Sydney 2015 *\n* Global Game Jam ""Best Audio"" Sydney 2015 *\n\nIt's the end of the world and you are the lone survivor in a tiny spaceship. Get ready to dodge everything that is trying to kill you! Your reaction time and precision is key! No Humanity is the hardest bullet hell dodge game. Compare your score with friends and watch as..."
228,Bus Rush 2,"Bus Rush 2 is one of the most complete multiplayer runners for Android. \nRun along Rio de Janeiro and other scenarios. Drag to jump or slide and to move left or right, avoid hitting obstacles like trucks, buses and subway trains among others!\nPlay races with other users around the world in the multiplayer mode. Run around and gather all the coins you can in different scenarios from Rio city like downtown, subway, sewer, forest, different beaches, and an amazing jungle!\n\nIn Bus Rush 2, yo..."
1465,Virus War - Space Shooting Game,"Warning! Virus invasion! Destroy them with your fingertip! \nis a free casual shooting game. Using only your fingertip, destroy all sorts of viruses. Remember to dodge, don’t let those filthy things hit your ship!\n*Simple and engaging gameplay. Play Virus War anywhere and anytime; get the most fun out of your breaks!\n*Equip your ship with different weapons and blast through swarms of enemies!\n*Surpass your friends in the ranking; set new records!"
360,Dancing Road: Color Ball Run!,Try out the most exciting Running - Sliding - Matching Music Game!\n\nThe rolling ball starts simply and ramps up shortly. \n\n★ Hold and drag your rolling ball to match other balls of the same color!\n★ Dodge different color balls!\n★ Try to collect all the coins and Gift Boxes on the dancing road!\n\nEnjoy the catchy music and challenges designed for each dancing road. \n\nLet's roll the ball and feel the beat in this Color Matching Game!
184,Bob - jigsaw puzzles free games for kids & parents,"Free jigsaw puzzles for kids, hundreds of puzzles for toddlers to assemble. Try now kids puzzle games for toddlers.\nJigsaw puzzles are great game for your toddler to play in waiting room or anywhere while you have to wait.\n\nFeatures:\n- kid puzzle game for free\n- unlimited number of pictures, many colorful pictures to choose from by your children\n- Perfect kid game when you 

## Problem 3: Custom preprocessing & tokenization

In Problem 1, you should have seen that `TfidfVectorizer` already performs some preprocessing by default and also does its own tokenization of the input data. This is great for getting started, but often we want to have more control over these steps. We can customize some aspects of the preprocessing through arguments when instantiating `TfidfVectorizer`, but for this exercise, we want to do _all_ of our preprocessing & tokenizing outside of scikit-learn.

Concretely, we want to use [spaCy](https://spacy.io), a library that we will make use of in later labs as well.  Here is a brief example of how to load and use a spaCy model:

In [12]:
import spacy
# Load the small English model, disabling some components that we don't need right now
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'textcat'])

# Take an example sentence and print every token from it separately
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")
for token in doc:
    print(token.text)

Apple
is
looking
at
buying
U.K.
startup
for
$
1
billion


**Your task** is to write a preprocessing function that uses spaCy to perform the following steps:
- tokenization
- lemmatization
- stop word removal
- removing tokens containing non-alphabetical characters

We recommend that you go through the [Linguistic annotations](https://spacy.io/usage/spacy-101#annotations) section of the spaCy&nbsp;101, which demonstrates how you can get the relevant kind of information via the spaCy library.

Implement your preprocessor by completing the following function:

In [13]:
def preprocess(text):
    """Preprocess the given text by tokenising it, removing any stop words, 
    replacing each remaining token with its lemma (base form), and discarding 
    all lemmas that contain non-alphabetical characters.

    Arguments:
      text (str): The text to preprocess.

    Returns:
      The list of remaining lemmas after preprocessing (represented as strings).
    """
    # YOUR CODE HERE
    # https://spacy.io/usage/spacy-101#annotations
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc if token.is_stop == False and token.is_alpha==True]
    return(lemmas)
    # raise NotImplementedError()

### 🤞 Test your code

Test your implementation by running the following cell:

In [14]:
"""Check that the preprocessing returns the correct output for a number of test cases."""

assert (
    preprocess('Apple is looking at buying U.K. startup for $1 billion') ==
    ['Apple', 'look', 'buy', 'startup', 'billion']
)
assert (
    preprocess('"Love Story" is a country pop song written and sung by Taylor Swift.') ==
    ['Love', 'Story', 'country', 'pop', 'song', 'write', 'sing', 'Taylor', 'Swift']
)
success()

## Problem 4: The effect of preprocessing

To make use of the new `preprocess` function from Problem 3, we need to make sure that we incorporate it into `TfidfVectorizer` and disable all preprocessing & tokenization that `TfidfVectorizer` performs by default. Afterwards, we also need to re-fit the vectorizer and the nearest-neighbors class. To make this a bit easier to handle, let’s take everything we have done so far and put it in a single class `AppSearcher`.

### Task 4.1

**Your first task** is to complete the stub of the `AppSearcher` class given below. Keep in mind:
- The `fit()` function should fit both the vectorizer (from Problem 1) and the nearest-neighbors class (from Problem 2).  Make sure to modify the call to `TfidfVectorizer` to _disable all preprocessing & tokenization_ that it would do by default, and replace it with a call to the `preprocess()` function _defined in `AppSearcher`_.
- For the `preprocess()` function, you can start by copying your solution from Problem 3.
- For the `search()` function, you can copy your solution from Problem 2.
- Make sure to adapt your code to store the everything (data, vectorizer, nearest-neighbors class) within the `AppSearcher` class, so that your solution is independent of the code you wrote above!

In [15]:
class AppSearcher:
    def fit(self, df):
        """Instantiate and fit all the classes required for the search engine (cf. Problems 1 and 2)."""
        self.df = df
        # YOUR CODE HERE
        # https://stackoverflow.com/questions/41988915/avoiding-loading-spacy-data-in-each-subprocess-when-multiprocessing
        self.nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'textcat'])
        # https://www.davidsbatista.net/blog/2018/02/28/TfidfVectorizer/
        self.vectorizer = TfidfVectorizer(tokenizer=self.preprocess, preprocessor=lambda x: x, token_pattern=None)
        self.X = self.vectorizer.fit_transform(df['description'])
        self.neighbors_nearest = NearestNeighbors(n_neighbors=10)
        self.neighbors_nearest.fit(self.X)
        # raise NotImplementedError()

    def preprocess(self, text):
        """Preprocess the given text (cf. Problem 3)."""
        # YOUR CODE HERE
        # self.nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'textcat'])
        doc = self.nlp(text)
        lemmas = [token.lemma_ for token in doc if token.is_stop == False and token.is_alpha==True]
        return(lemmas)
        # raise NotImplementedError()

    def search(self, query):
        """Find the nearest neighbors in `df` for a query string (cf. Problem 2)."""
        # YOUR CODE HERE
        query_vector = self.vectorizer.transform([query])
        # https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html
        neigh_dist, neigh_ind = self.neighbors_nearest.kneighbors(query_vector)
        name = self.df.iloc[neigh_ind[0]]['name']
        description = self.df.iloc[neigh_ind[0]]['description']
        distance = neigh_dist[0]
        return pd.DataFrame({'name': name, 'distance': distance, "description" : description})
        # raise NotImplementedError()


#### 🤞 Test your code

The following cell demonstrates how your class should be used. Note that it can take a bit longer to train it on the data as before, since we’re now calling spaCy for the preprocessing.

In [16]:
apps = AppSearcher()
apps.fit(df)
apps.search("pile up pancakes")

,name,distance,description
1032,Pancake Tower,0.290758,Let's see how many pancakes you can pile up!!
326,Cooking School: Games for Girls,1.264094,"Children like to help their parents. They especially like to help with cooking . When there is a cooking in the kitchen, it is no way to play. But cooking is a complicated process and often it ends up with a huge mess in the kitchen. But what if you are so eager to cook pancakes, cake or cupcakes? How to cook all that without doing a cleaning after? We have a solution! Home Cooking School with our curious Hippo has opened especially for parents and children! We do not only cook food here. We..."
1235,Solitaire,1.331677,"Solitaire Free by Solitaire Card Games is the #1 klondike solitaire games on android. The solitaire Free is popular and classic card games you know and love.\n\nWe carefully designed a fresh solitaire free modern look, woven into the wonderful solitaire classic feel that everyone loves. \n\nExperience the crisp, clear, and easy to read cards, simple and quick animations, and subtle sounds, in either landscape or portrait views. \n\nYou can move cards with a single tap or drag them to their d..."
1181,Sago Mini Trucks and Diggers,1.339758,"Drive a dump truck with Rosie the hamster! Pile dirt high and dig deep in the ground with diggers, cranes and bulldozers. Build a home for a new friend! Choose a barn, a castle or even a cupcake-house. Don’t forget to add the finishing touches for the proud owner.\n\nOn this construction site, kids love being the boss. With six mighty machines and piles of dirt, you can build all day! Part of the award-winning suite of Sago Mini apps, this app puts kids in charge.\n\nSago Mini apps have no i..."
1263,Spider Solitaire,1.345319,"Spider Solitaire was built to offer card players a fun way to play their favorite classic in both portrait and landscape mode.\n\nWith large cards and a unique stacking system our Spider card game doesn't have problems fitting your screen like many others do. \n\n* How to play *\n\nTo win a game of spider solitaire, all cards must be removed from the table. Assembling the cards in the tableau allows for cards to be placed in their respective stacks in order. At the beginning of each game, 54..."
656,"Hell’s Cooking — crazy chef burger, kitchen fever",1.363271,"⭐ ⭐ ⭐ ⭐ ⭐ New world of crazy cooking is here. Feel what it means to be a master chef who prepares fantastic fast food in a prominent king kitchen! If you haven't ever tried yourself as a hamburger chef cook, it's possibly the best time for making diner. Download and launch Hell's Cooking — crazy chef burger, kitchen fever HD game and get prepared to jump into a fever and adventurous perfect world of burgers.\n\nNew girls game Hell's Cooking gives you lots of opportunities for your crazy cafe..."
1164,Rummy - Free,1.372016,"Play the famous Rummy card game on your Android Smartphone or Tablet !! \n\nPlay rummy with 2, 3, or 4 players against simulated opponents playing with high-level artificial intelligence. \nThere are a number of rules that can be modified, making this game very faithful to the original. \n\n*** MANY VARIATIONS INCLUDED *** \n\nMany rummy variations are included in the application: \n\n- From 2 to 4 players. \n- Choose the AI level of opponents. \n- Number of cards dealt to each player (from ..."
436,Dr. Panda's Ice Cream Truck,1.378334,"Chocolate? Vanilla? Strawberry? All three!? You decide! In Dr. Panda’s Ice Cream Truck you can mix up all sorts of different flavors with cookies, chocolate, nuts and more to make the perfect ice cream—hundreds of combinations in all.\n\nScoop it!\nThese animals love ice cream, and will eat as much (or little) as you want to serve them. You can make scoops big or small and pile them as high as you want—using any of the ice cream you’ve created!\n\nToppings galore!\nUse chocolate syrup, cooki..."
1245,Solitaire Free,1.380202,"Solitaire by Gemego is the card game you know and love for your phone and table

### Task 4.2

**Your second task** is to experiment with the effect of using (or not using) different preprocessing steps.  We always need to _tokenize_ the text, but other preprocessing steps are optional and require a conscious decision whether to use them or not, such as:
- lemmatization
- lowercasing all characters
- removing stop words
- removing tokens containing non-alphabetical characters

**Modify the definition of the `preprocess()` function** of `AppSearcher` to include/exclude individual preprocessing steps, run some searches, and observe if and how the results change.  Which search queries you try out is up to you — you could compare searching for "pile up pancakes" with "pancake piling", for example; or you could try entirely different search queries aimed at different kinds of apps.  (You can modify the class directly by changing the cell above under Task 4.1, or copy the definitions to the cells below, whichever you prefer; there is no separate code to show for this task, but you will use your observations here for the individual reflection.)

In [17]:
# lemmatization
class AppSearcher:
    def fit(self, df):
        """Instantiate and fit all the classes required for the search engine (cf. Problems 1 and 2)."""
        self.df = df
        # YOUR CODE HERE
        # https://stackoverflow.com/questions/41988915/avoiding-loading-spacy-data-in-each-subprocess-when-multiprocessing
        self.nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'textcat'])
        # https://www.davidsbatista.net/blog/2018/02/28/TfidfVectorizer/
        self.vectorizer = TfidfVectorizer(tokenizer=self.preprocess, preprocessor=lambda x: x, token_pattern=None)
        self.X = self.vectorizer.fit_transform(df['description'])
        self.neighbors_nearest = NearestNeighbors(n_neighbors=10)
        self.neighbors_nearest.fit(self.X)
        # raise NotImplementedError()

    def preprocess(self, text):
        """Preprocess the given text (cf. Problem 3)."""
        # YOUR CODE HERE
        # self.nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'textcat'])
        doc = self.nlp(text)
        lemmas = [token.lemma_ for token in doc]
        return(lemmas)
        # raise NotImplementedError()

    def search(self, query):
        """Find the nearest neighbors in `df` for a query string (cf. Problem 2)."""
        # YOUR CODE HERE
        query_vector = self.vectorizer.transform([query])
        # https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html
        neigh_dist, neigh_ind = self.neighbors_nearest.kneighbors(query_vector)
        name = self.df.iloc[neigh_ind[0]]['name']
        description = self.df.iloc[neigh_ind[0]]['description']
        distance = neigh_dist[0]
        return pd.DataFrame({'name': name, 'distance': distance, "description" : description})
        # raise NotImplementedError()

In [18]:
apps = AppSearcher()
apps.fit(df)
apps.search("pile up pancakes")

,name,distance,description
1032,Pancake Tower,0.662227,Let's see how many pancakes you can pile up!!
326,Cooking School: Games for Girls,1.289065,"Children like to help their parents. They especially like to help with cooking . When there is a cooking in the kitchen, it is no way to play. But cooking is a complicated process and often it ends up with a huge mess in the kitchen. But what if you are so eager to cook pancakes, cake or cupcakes? How to cook all that without doing a cleaning after? We have a solution! Home Cooking School with our curious Hippo has opened especially for parents and children! We do not only cook food here. We..."
1235,Solitaire,1.347524,"Solitaire Free by Solitaire Card Games is the #1 klondike solitaire games on android. The solitaire Free is popular and classic card games you know and love.\n\nWe carefully designed a fresh solitaire free modern look, woven into the wonderful solitaire classic feel that everyone loves. \n\nExperience the crisp, clear, and easy to read cards, simple and quick animations, and subtle sounds, in either landscape or portrait views. \n\nYou can move cards with a single tap or drag them to their d..."
1181,Sago Mini Trucks and Diggers,1.354141,"Drive a dump truck with Rosie the hamster! Pile dirt high and dig deep in the ground with diggers, cranes and bulldozers. Build a home for a new friend! Choose a barn, a castle or even a cupcake-house. Don’t forget to add the finishing touches for the proud owner.\n\nOn this construction site, kids love being the boss. With six mighty machines and piles of dirt, you can build all day! Part of the award-winning suite of Sago Mini apps, this app puts kids in charge.\n\nSago Mini apps have no i..."
1263,Spider Solitaire,1.354457,"Spider Solitaire was built to offer card players a fun way to play their favorite classic in both portrait and landscape mode.\n\nWith large cards and a unique stacking system our Spider card game doesn't have problems fitting your screen like many others do. \n\n* How to play *\n\nTo win a game of spider solitaire, all cards must be removed from the table. Assembling the cards in the tableau allows for cards to be placed in their respective stacks in order. At the beginning of each game, 54..."
656,"Hell’s Cooking — crazy chef burger, kitchen fever",1.373068,"⭐ ⭐ ⭐ ⭐ ⭐ New world of crazy cooking is here. Feel what it means to be a master chef who prepares fantastic fast food in a prominent king kitchen! If you haven't ever tried yourself as a hamburger chef cook, it's possibly the best time for making diner. Download and launch Hell's Cooking — crazy chef burger, kitchen fever HD game and get prepared to jump into a fever and adventurous perfect world of burgers.\n\nNew girls game Hell's Cooking gives you lots of opportunities for your crazy cafe..."
1164,Rummy - Free,1.381234,"Play the famous Rummy card game on your Android Smartphone or Tablet !! \n\nPlay rummy with 2, 3, or 4 players against simulated opponents playing with high-level artificial intelligence. \nThere are a number of rules that can be modified, making this game very faithful to the original. \n\n*** MANY VARIATIONS INCLUDED *** \n\nMany rummy variations are included in the application: \n\n- From 2 to 4 players. \n- Choose the AI level of opponents. \n- Number of cards dealt to each player (from ..."
1442,Turbo Dismount™,1.383333,"The legendary crash simulator is now on Google Play!\n\nPerform death-defying motor stunts, crash into walls, create traffic pile-ups of epic scale - and share the fun!\n\nTurbo Dismount™ is a kinetic tragedy about Mr. Dismount and the cars who love him. It is the official sequel to the wildly popular and immensely successful personal impact simulator - Stair Dismount™. \n\nFEATURES:\n* Flinch-inducing crash physics\n* Crunchy sound effects\n* Delicious slow-mo replay system\n* Multiple vehi..."
436,Dr. Panda's Ice Cream Truck,1.383684,"Chocolate? Vanilla? Strawberry? All three!? You decide! In Dr. Panda’s Ice Cre

In [19]:
apps.search("pancake piling")

,name,distance,description
1032,Pancake Tower,0.889836,Let's see how many pancakes you can pile up!!
326,Cooking School: Games for Girls,1.253859,"Children like to help their parents. They especially like to help with cooking . When there is a cooking in the kitchen, it is no way to play. But cooking is a complicated process and often it ends up with a huge mess in the kitchen. But what if you are so eager to cook pancakes, cake or cupcakes? How to cook all that without doing a cleaning after? We have a solution! Home Cooking School with our curious Hippo has opened especially for parents and children! We do not only cook food here. We..."
656,"Hell’s Cooking — crazy chef burger, kitchen fever",1.360802,"⭐ ⭐ ⭐ ⭐ ⭐ New world of crazy cooking is here. Feel what it means to be a master chef who prepares fantastic fast food in a prominent king kitchen! If you haven't ever tried yourself as a hamburger chef cook, it's possibly the best time for making diner. Download and launch Hell's Cooking — crazy chef burger, kitchen fever HD game and get prepared to jump into a fever and adventurous perfect world of burgers.\n\nNew girls game Hell's Cooking gives you lots of opportunities for your crazy cafe..."
1601,من سيربح المليون الموسوعة,1.414214,برنامج المسابقات الشهير (من سيربح المليون) بالنسخة الأصلية للبرنامج ...\nالإصدار الذهبي الموسوعة ميزاته كثيرة نذكر منها :\n\n1- إضافة أكثر من 19 آلف سؤال ليتحول البرنامج من لعبة إلى موسوعة حقيقية\n2- اعتماد نظام الأسئلة المتدرجة في الصعوبة ليصبح الإصدار الذهبي نسخة عن البرنامج التلفزيوني الشهير الذي يحمل نفس الاسم (من سيربح المليون)\n3- إضافة وسيلة المساعدة الجديدة (إمكانية تغيير السؤال بعد السؤال الخامس) \n4- إضافة وسيلة مساعدة أخرى وجديدة (بعد الوصول للسؤال السابع) وهي طلب مساعدة احد أصدقا...
1592,Сканворды на русском,1.414214,"""Сканворды на русском""- интереснейшая словесная головоломка, которая не заставит Вас скучать. От создателей Быстрых кроссвордов. Еще больше пересечений слов. Еще проще, веселее и удобнее.\n\nИгра проста в использовании и очень забавна. Решая ""Сканворды на русском"", Вы освежаете свои знания, расширяете словарный запас, тренируете мозг и развиваете интеллект.\n\nСканворды на русском обладают всеми перечисленными признаками: \n• Логическая игра- развлечение, которое стимулирует ум.\n• Головолом..."
1600,كلمات وصور - كلمة السر الذكية 3,1.414214,نقدم لكم العاب بريك لعبة الذكاء الجديدة والشيقة كلمات وصور بريك وصلة لعبة باسلوب فريد وممتع تناسب جميع افراد العائلة وتقضي معها اجمل الاوقات \nهذه اللعبة جديدة كلياً مبنية على فكرة الكلمات المتقاطعة وكلمة السر ولكن بصورة عصرية ومسلية حيث قمنا بوضع الغاز في كل مرحلة وعليك البحث وفك رموز هذه الالغاز للوصل الى حل اللغز .. جربها !!\n\nكلمات وصور - كلمة السر الذكية 3 مبنية على لعبة كلمة السر ولكن هنا سوف تقوم بالبحث عن معاني الرموز بدلا من البحث عن الكلمات حسب تقييم المركز العالمي للبحوث ان الصور...
1212,Skat,1.414214,Spiele Skat wann und wo Du willst!\nStarke Gegner. Erstklassiges Design.\n\nSpiele jederzeit gegen starke Computergegner!\n\nWir bieten ein umfangreiches Trainings-Angebot für Skatspieler und für alle die es werden wollen.\nFreue Dich auf viele Stunden Spielspaß!\n\nSpiele gegen starke Computerspieler:\n- Spiele überall und jederzeit offline Skat\n- Einstellbare Spielstärke\n- Deine Computergegner spielen 100% fair\n\nLöse Rätsel von den besten Skatspielern der Welt:\n- Teste Dein Können im ...
1611,이사만루2019,1.414214,"[2019년 8월 업데이트]\n- 신규 레전드 카드 추가\n- 메인 화면 이벤트 시스템 기능 개선\n- 아이템 낱개 개봉 기능 추가\n- 기타 버그 수정\n\n[2019년 6월 업데이트]\n- 신규 콘텐츠 ‘서바이벌 대전’ 추가\n- 실시간 대전 ‘현상금 시스템’ 추가\n- 코치 종류(클래식, 내셔널) 변경 기능 추가\n- 코치&장비팩 바로 먹이기 기능 추가\n- 다른 유저 로스터 복사 기능 추가\n- 교환소 기능 개선\n\n[2019년 4월 업데이트]\n- 연속 출석 시스템 추가\n- 관전 모드 Pitch Zone 추가\n- 이적 시장 편의성 개선\n- 공 궤적 표시 옵션화\n- 확정 강화 UI 개선\n\n[2019년 3월 대규모 업데이트]\n- 선수 루틴(사전 동작) 추가\n- 90~93 연도 선수 추가\n- 한계 돌파 시스템 추가\n(레벨/컨디션 상향/유저 구단명 각인 가능)\n- 선수 카드 통산 기록 추가\n- 랭커/친구/클럽원 플레이 관전 모드 추가 \n\n\n▶ 필..."
1599,دردشاتي - تعارف شات و زواج,1.414214,"تطبيق دردشاتي هو أضخم وأكبر تطبيق تعارف عربي في العالم العربي !\n\nدردشا

In [20]:
# lowercasing all characters
class AppSearcher:
    def fit(self, df):
        """Instantiate and fit all the classes required for the search engine (cf. Problems 1 and 2)."""
        self.df = df
        # YOUR CODE HERE
        # https://stackoverflow.com/questions/41988915/avoiding-loading-spacy-data-in-each-subprocess-when-multiprocessing
        self.nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'textcat'])
        # https://www.davidsbatista.net/blog/2018/02/28/TfidfVectorizer/
        self.vectorizer = TfidfVectorizer(tokenizer=self.preprocess, preprocessor=lambda x: x, token_pattern=None)
        self.X = self.vectorizer.fit_transform(df['description'])
        self.neighbors_nearest = NearestNeighbors(n_neighbors=10)
        self.neighbors_nearest.fit(self.X)
        # raise NotImplementedError()

    def preprocess(self, text):
        """Preprocess the given text (cf. Problem 3)."""
        # YOUR CODE HERE
        # self.nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'textcat'])
        doc = self.nlp(text)
        # https://spacy.io/api/token
        lower_case = [token.text.lower() for token in doc]
        return(lower_case)
        # raise NotImplementedError()

    def search(self, query):
        """Find the nearest neighbors in `df` for a query string (cf. Problem 2)."""
        # YOUR CODE HERE
        query_vector = self.vectorizer.transform([query])
        # https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html
        neigh_dist, neigh_ind = self.neighbors_nearest.kneighbors(query_vector)
        name = self.df.iloc[neigh_ind[0]]['name']
        description = self.df.iloc[neigh_ind[0]]['description']
        distance = neigh_dist[0]
        return pd.DataFrame({'name': name, 'distance': distance, "description" : description})
        # raise NotImplementedError()

In [21]:
apps = AppSearcher()
apps.fit(df)
apps.search("pile up pancakes")

,name,distance,description
1032,Pancake Tower,0.588768,Let's see how many pancakes you can pile up!!
326,Cooking School: Games for Girls,1.295255,"Children like to help their parents. They especially like to help with cooking . When there is a cooking in the kitchen, it is no way to play. But cooking is a complicated process and often it ends up with a huge mess in the kitchen. But what if you are so eager to cook pancakes, cake or cupcakes? How to cook all that without doing a cleaning after? We have a solution! Home Cooking School with our curious Hippo has opened especially for parents and children! We do not only cook food here. We..."
656,"Hell’s Cooking — crazy chef burger, kitchen fever",1.373826,"⭐ ⭐ ⭐ ⭐ ⭐ New world of crazy cooking is here. Feel what it means to be a master chef who prepares fantastic fast food in a prominent king kitchen! If you haven't ever tried yourself as a hamburger chef cook, it's possibly the best time for making diner. Download and launch Hell's Cooking — crazy chef burger, kitchen fever HD game and get prepared to jump into a fever and adventurous perfect world of burgers.\n\nNew girls game Hell's Cooking gives you lots of opportunities for your crazy cafe..."
1235,Solitaire,1.377988,"Solitaire Free by Solitaire Card Games is the #1 klondike solitaire games on android. The solitaire Free is popular and classic card games you know and love.\n\nWe carefully designed a fresh solitaire free modern look, woven into the wonderful solitaire classic feel that everyone loves. \n\nExperience the crisp, clear, and easy to read cards, simple and quick animations, and subtle sounds, in either landscape or portrait views. \n\nYou can move cards with a single tap or drag them to their d..."
1164,Rummy - Free,1.380235,"Play the famous Rummy card game on your Android Smartphone or Tablet !! \n\nPlay rummy with 2, 3, or 4 players against simulated opponents playing with high-level artificial intelligence. \nThere are a number of rules that can be modified, making this game very faithful to the original. \n\n*** MANY VARIATIONS INCLUDED *** \n\nMany rummy variations are included in the application: \n\n- From 2 to 4 players. \n- Choose the AI level of opponents. \n- Number of cards dealt to each player (from ..."
436,Dr. Panda's Ice Cream Truck,1.382481,"Chocolate? Vanilla? Strawberry? All three!? You decide! In Dr. Panda’s Ice Cream Truck you can mix up all sorts of different flavors with cookies, chocolate, nuts and more to make the perfect ice cream—hundreds of combinations in all.\n\nScoop it!\nThese animals love ice cream, and will eat as much (or little) as you want to serve them. You can make scoops big or small and pile them as high as you want—using any of the ice cream you’ve created!\n\nToppings galore!\nUse chocolate syrup, cooki..."
1181,Sago Mini Trucks and Diggers,1.383154,"Drive a dump truck with Rosie the hamster! Pile dirt high and dig deep in the ground with diggers, cranes and bulldozers. Build a home for a new friend! Choose a barn, a castle or even a cupcake-house. Don’t forget to add the finishing touches for the proud owner.\n\nOn this construction site, kids love being the boss. With six mighty machines and piles of dirt, you can build all day! Part of the award-winning suite of Sago Mini apps, this app puts kids in charge.\n\nSago Mini apps have no i..."
1326,TO-FU Oh!SUSHI,1.384821,"You are the veritable sushi master! Prepare your own fun sushi with “Daizu” the skunk!\n\nThis app is designed to allow children to be creative by decorating their original sushi.\n\nServe your delicious, mysterious or impossible sushi to the people of “Tofu Island”! \n\nHow about creating sushi that is totally original and serve it to your beloved guests? Spice it up with tons of wasabi or even sprinkle chocolate and gummy bears for those sweet lovers.\nFeel free to make any kind of sushi y..."
1442,Turbo Dismount™,1.385302,"The legendary crash simulator is now on Google Play!\n\nPerform death-defying mo

In [22]:
apps.search("pancake piling")

,name,distance,description
1298,Stylist Girl - Make Me Gorgeous!,1.379455,"If you’re into makeover games, this one will knock your socks off. And if you’re not into makeover games, this one will change that! \n\nYou’re the hottest new fashion stylist girl in Hollywood, and your awesome reputation is already bringing you fame and fortune! Dress up famous fashion models and VIP celebrities. Bring out their star style and give them beautiful makeovers! Create tons of incredible looks for your clients - find them the perfect outfits for their big events - they’ll love ..."
1610,기적의 검,1.414214,"소지섭, 안젤리나가 선택한 바로 그 게임!\n\n★ 공식카페에서 구글 사전예약 한정 특별 선물 배포중! ★\n\n\n이 세상 편함이 아니다!\n세로의 치명적인 매력\n세로운 한손 액션 판타지 rpg 기적의 검\n\n\n1. 이보다 편한 RPG는 없다!\n- 한손 원터치 폭풍 성장과 \n끊임없는 오프라인 성장으로 시간을 지배하라!\n\n2. 연맹 영지 탈환을 위한 핏빛 전쟁!\n- 각 영주 자리를 향한 대규모 연맹의 아찔한 쟁패가 시작된다!\n\n3. 여러가지맛 성장 시스템\n- 다양한 개성을 녹여 지루할 틈이 없는 \n성령 / 신병 / 마법진 / 신수 등 다채로운 육성법으로\n무한한 성장의 기쁨을 느껴보세요!\n\n4. 너와 나의 만남은 우연이 아니다!\n- 한 손으로 맺는 아름다운 인연, \n또 하나의 소중한 인연이 당신만을 기다린다.\n\n5. 모으고 꾸미고 성장하라!\n- 화려하고 개성넘치는 다양한 코스튬의 파도속에서\n나만의 개성을 찾아 거침없이 개방하라!\n\n▶기적..."
1167,RusDate - Dating. Знакомства и Общение на Русском,1.414214,"Скачай бесплатно лучший чат знакомств на русском языке RusDate и общайся😉. Найти парня или девушку в активном поиске рядом с тобой, или новых друзей - запросто: взаимный лайк на анкету и вы строите отношения вдвоем 💕. Заметил, что рядом симпатичная незнакомка - действуй🎯. Открывай программу, выбирай «Ищу женщину», ставь пометку «рядом со мной» и смотри анкеты девушек, которые поблизости с точностью до метра. Незнакомка может превратиться в твою девушку и жену👰, незнакомец - в парня и мужа🤵. ..."
1599,دردشاتي - تعارف شات و زواج,1.414214,"تطبيق دردشاتي هو أضخم وأكبر تطبيق تعارف عربي في العالم العربي !\n\nدردشاتي الذي يشمل أكثر من مليون مستخدم يبحثون عن صداقة تعارف وزواج بالاضافة الى زواج مسيار، وهو تطبيق التعارف العربي الأكبر والاضخم بعدد مسجلين حقيقيين من انحاء العالم العربي .\n\nنقدم لكم تطبيق التعارف الاقوى والاشهر في الشرق الاوسط تطبيق ""دردشاتي""، والمعد بصورة خاصة للمستخدمين العرب، حيث يشمل التطبيق مئات الاف المشتركين ، والمئات الجدد يوميا من شتى انحاء العالم العربي .\n\nالتطبيق تم برمجته وتصميمه بأفضل النواحي التقنية حيث..."
1611,이사만루2019,1.414214,"[2019년 8월 업데이트]\n- 신규 레전드 카드 추가\n- 메인 화면 이벤트 시스템 기능 개선\n- 아이템 낱개 개봉 기능 추가\n- 기타 버그 수정\n\n[2019년 6월 업데이트]\n- 신규 콘텐츠 ‘서바이벌 대전’ 추가\n- 실시간 대전 ‘현상금 시스템’ 추가\n- 코치 종류(클래식, 내셔널) 변경 기능 추가\n- 코치&장비팩 바로 먹이기 기능 추가\n- 다른 유저 로스터 복사 기능 추가\n- 교환소 기능 개선\n\n[2019년 4월 업데이트]\n- 연속 출석 시스템 추가\n- 관전 모드 Pitch Zone 추가\n- 이적 시장 편의성 개선\n- 공 궤적 표시 옵션화\n- 확정 강화 UI 개선\n\n[2019년 3월 대규모 업데이트]\n- 선수 루틴(사전 동작) 추가\n- 90~93 연도 선수 추가\n- 한계 돌파 시스템 추가\n(레벨/컨디션 상향/유저 구단명 각인 가능)\n- 선수 카드 통산 기록 추가\n- 랭커/친구/클럽원 플레이 관전 모드 추가 \n\n\n▶ 필..."
1595,Филворды: поиск слов,1.414214,"Игра Филворды - это несколько сотен уровней интеллектуального наслаждения. Вы ищете загаданные слова, а симпатичные монстры помогают вам. Сможете ли вы дойти до конца игры, заполнив все квадраты словами, а сосуды наполнить разноцветной жидкостью? \n\nЦелая куча уровней, спрятанных в бутылках!\n\nБолее 1000 уровней подготовлены специально для вас. В начале разгадывание головоломки может показаться несложным занятием. Но, по мере того, как количество букв увеличивается, угадывать слова станови..."
1604,ดัมมี่ - Dummy มีไพ่แคง ป๊อกเด้ง,1.414214,ไพ่ดัมมี่ หรือ Dummy ไพ่ดำมี่ รัมมี่ไทย เป็นเกมไพ่ยอดนิยมในเมืองไทย จำเป็นต้องอาศัยทักษะและความจำด้วย แค่ติดตั้งแอพฯนี้ ก็จะเล่นไพ่ไทยเสมือนจริงดัมมี่ Dummy ไพ่แคงไทย ป๊อกเด้ง เก้าเกไทย ไพ่ผสมสิบ น้ำเต้า ปู ปลาได้เลย! นอกจากนี้ คุณยังสามารถเล่นสล็อต วงล้อได้ 24 ชั่วโมงจร้า! \n★จุดเด่นดึงดูดสายตา !★\n♠ ฟรี! โหลดฟรี! เล่นฟรี! เล่นแล้วแก้เครียด แก้เหงา!\n♠ ช่องทางได้ชิปฟรีมีหลากหลาย ล็อกอินต่อเนื่องรับชิปฟรีเป็นแสน\n♠ ทีมงานจัดกิจกรรมสนุกสนานในเกมส์ประจำ เช่นโปรโมชั่นแถมฟรี ทุบไข่ เปิดจิ๊กซอว์ ...
686,Hoşkin HD - İnternetsiz,1.414214,"Hoşkin HD, batak ve king oyununa benzeyen bir kart oyunudur.\n\nÖzellikler\n-3 kişilik oyun modu\n-Internet bağlantısına gerek olmadan istediğin

In [23]:
# removing stop words
class AppSearcher:
    def fit(self, df):
        """Instantiate and fit all the classes required for the search engine (cf. Problems 1 and 2)."""
        self.df = df
        # YOUR CODE HERE
        # https://stackoverflow.com/questions/41988915/avoiding-loading-spacy-data-in-each-subprocess-when-multiprocessing
        self.nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'textcat'])
        # https://www.davidsbatista.net/blog/2018/02/28/TfidfVectorizer/
        self.vectorizer = TfidfVectorizer(tokenizer=self.preprocess, preprocessor=lambda x: x, token_pattern=None)
        self.X = self.vectorizer.fit_transform(df['description'])
        self.neighbors_nearest = NearestNeighbors(n_neighbors=10)
        self.neighbors_nearest.fit(self.X)
        # raise NotImplementedError()

    def preprocess(self, text):
        """Preprocess the given text (cf. Problem 3)."""
        # YOUR CODE HERE
        # self.nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'textcat'])
        doc = self.nlp(text)
        rm_stop_word = [token.text for token in doc if token.is_stop == False]
        return(rm_stop_word)
        # raise NotImplementedError()

    def search(self, query):
        """Find the nearest neighbors in `df` for a query string (cf. Problem 2)."""
        # YOUR CODE HERE
        query_vector = self.vectorizer.transform([query])
        # https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html
        neigh_dist, neigh_ind = self.neighbors_nearest.kneighbors(query_vector)
        name = self.df.iloc[neigh_ind[0]]['name']
        description = self.df.iloc[neigh_ind[0]]['description']
        distance = neigh_dist[0]
        return pd.DataFrame({'name': name, 'distance': distance, "description" : description})
        # raise NotImplementedError()


In [24]:
apps = AppSearcher()
apps.fit(df)
apps.search("pile up pancakes")

,name,distance,description
1032,Pancake Tower,0.429131,Let's see how many pancakes you can pile up!!
326,Cooking School: Games for Girls,1.274510,"Children like to help their parents. They especially like to help with cooking . When there is a cooking in the kitchen, it is no way to play. But cooking is a complicated process and often it ends up with a huge mess in the kitchen. But what if you are so eager to cook pancakes, cake or cupcakes? How to cook all that without doing a cleaning after? We have a solution! Home Cooking School with our curious Hippo has opened especially for parents and children! We do not only cook food here. We..."
656,"Hell’s Cooking — crazy chef burger, kitchen fever",1.368825,"⭐ ⭐ ⭐ ⭐ ⭐ New world of crazy cooking is here. Feel what it means to be a master chef who prepares fantastic fast food in a prominent king kitchen! If you haven't ever tried yourself as a hamburger chef cook, it's possibly the best time for making diner. Download and launch Hell's Cooking — crazy chef burger, kitchen fever HD game and get prepared to jump into a fever and adventurous perfect world of burgers.\n\nNew girls game Hell's Cooking gives you lots of opportunities for your crazy cafe..."
1235,Solitaire,1.375195,"Solitaire Free by Solitaire Card Games is the #1 klondike solitaire games on android. The solitaire Free is popular and classic card games you know and love.\n\nWe carefully designed a fresh solitaire free modern look, woven into the wonderful solitaire classic feel that everyone loves. \n\nExperience the crisp, clear, and easy to read cards, simple and quick animations, and subtle sounds, in either landscape or portrait views. \n\nYou can move cards with a single tap or drag them to their d..."
1164,Rummy - Free,1.380230,"Play the famous Rummy card game on your Android Smartphone or Tablet !! \n\nPlay rummy with 2, 3, or 4 players against simulated opponents playing with high-level artificial intelligence. \nThere are a number of rules that can be modified, making this game very faithful to the original. \n\n*** MANY VARIATIONS INCLUDED *** \n\nMany rummy variations are included in the application: \n\n- From 2 to 4 players. \n- Choose the AI level of opponents. \n- Number of cards dealt to each player (from ..."
436,Dr. Panda's Ice Cream Truck,1.380312,"Chocolate? Vanilla? Strawberry? All three!? You decide! In Dr. Panda’s Ice Cream Truck you can mix up all sorts of different flavors with cookies, chocolate, nuts and more to make the perfect ice cream—hundreds of combinations in all.\n\nScoop it!\nThese animals love ice cream, and will eat as much (or little) as you want to serve them. You can make scoops big or small and pile them as high as you want—using any of the ice cream you’ve created!\n\nToppings galore!\nUse chocolate syrup, cooki..."
1442,Turbo Dismount™,1.383131,"The legendary crash simulator is now on Google Play!\n\nPerform death-defying motor stunts, crash into walls, create traffic pile-ups of epic scale - and share the fun!\n\nTurbo Dismount™ is a kinetic tragedy about Mr. Dismount and the cars who love him. It is the official sequel to the wildly popular and immensely successful personal impact simulator - Stair Dismount™. \n\nFEATURES:\n* Flinch-inducing crash physics\n* Crunchy sound effects\n* Delicious slow-mo replay system\n* Multiple vehi..."
427,Dr. Panda Ice Cream Truck Free,1.387126,"Dr. Panda Ice Cream Truck is FREE for you to play!\n\nChocolate? Vanilla? Strawberry? All three!? You decide! In Dr. Panda Ice Cream Truck you can mix up all sorts of different flavors with cookies, chocolate, nuts and more to make the perfect ice cream—hundreds of combinations in all.\n\nScoop it!\nThese animals love ice cream, and will eat as much (or little) as you want to serve them. You can make scoops big or small and pile them as high as you want—using any of the ice cream you’ve crea..."
1245,Solitaire Free,1.389436,"Solitaire by Gemego is the card game you know and love for your phone and table

In [25]:
apps.search("pancake piling")

,name,distance,description
1298,Stylist Girl - Make Me Gorgeous!,1.375465,"If you’re into makeover games, this one will knock your socks off. And if you’re not into makeover games, this one will change that! \n\nYou’re the hottest new fashion stylist girl in Hollywood, and your awesome reputation is already bringing you fame and fortune! Dress up famous fashion models and VIP celebrities. Bring out their star style and give them beautiful makeovers! Create tons of incredible looks for your clients - find them the perfect outfits for their big events - they’ll love ..."
742,Kelimelik,1.414214,"Türkiye'nin en çok oynanan kelime oyunu 11 milyon üyeyi geçti! Bu başarıya ulaşmamızı sağlayan tüm oyuncularımıza teşekkür ediyoruz. Siz de Kelimelik ailesine katılmak ve arkadaşlarınızla Türkçe kelime oyunu keyfini yaşamak için hemen ücretsiz oyunumuzu indirin.\n\nOyunumuzun Özellikleri:\n\n★ Kelimelik her yerde! İster mobil cihazdan isterseniz de bilgisayarınızdan oynayabilirsiniz.\n\n★ Aynı anda 20'ye kadar oyun açabilir, bu sayede bekleme olmaksızın oyuna devam edebilirsiniz.\n\n★ Facebo..."
1601,من سيربح المليون الموسوعة,1.414214,برنامج المسابقات الشهير (من سيربح المليون) بالنسخة الأصلية للبرنامج ...\nالإصدار الذهبي الموسوعة ميزاته كثيرة نذكر منها :\n\n1- إضافة أكثر من 19 آلف سؤال ليتحول البرنامج من لعبة إلى موسوعة حقيقية\n2- اعتماد نظام الأسئلة المتدرجة في الصعوبة ليصبح الإصدار الذهبي نسخة عن البرنامج التلفزيوني الشهير الذي يحمل نفس الاسم (من سيربح المليون)\n3- إضافة وسيلة المساعدة الجديدة (إمكانية تغيير السؤال بعد السؤال الخامس) \n4- إضافة وسيلة مساعدة أخرى وجديدة (بعد الوصول للسؤال السابع) وهي طلب مساعدة احد أصدقا...
1612,컴투스프로야구2019,1.414214,★ 레전드 등급 추가로 더욱 강력해진 2019 메이저 업데이트 ★\n\n# 대한민국 레전드 야구게임! 컴투스프로야구2019 메이저 업데이트!\n- 모바일 야구 최강! 극강의 리얼리티를 자랑하는 컴프야 시리즈를 2019년에도 만나보세요!\n\n# 전설들의 귀환! 레전드 카드 출시\n- 무등산 폭격기 선동열! 철완 최동원! 적토마 이병규! 두목곰 김동주 등 KBO 최고의 선수들의 귀환\n- 레전드 카드를 통해 더욱 강력해진 선수들을 만나보세요!\n\n# 류현진이 등장하는 유일한 KBO 야구게임 컴프야!\n- 코리안 몬스터 류현진 선수를 게임에서 만나보세요.\n\n# 강한자가 살아남는다! 실시간 홈런 대전 홈런로얄 BETA 오픈\n- 하나의 타구가 승부를 좌우하는 실시간 홈런모드 신규 오픈\n- 홈런로얄 모드에서 홈런왕에 도전하세요!\n\n# 판타지 라인업 모드 추가!\n- 예측 야구의 끝판왕! \n- 판타지 라인업에서 실제 선수들의 기록을 예상하여 예측왕에 도전하세요!\n- 판타...
1592,Сканворды на русском,1.414214,"""Сканворды на русском""- интереснейшая словесная головоломка, которая не заставит Вас скучать. От создателей Быстрых кроссвордов. Еще больше пересечений слов. Еще проще, веселее и удобнее.\n\nИгра проста в использовании и очень забавна. Решая ""Сканворды на русском"", Вы освежаете свои знания, расширяете словарный запас, тренируете мозг и развиваете интеллект.\n\nСканворды на русском обладают всеми перечисленными признаками: \n• Логическая игра- развлечение, которое стимулирует ум.\n• Головолом..."
1386,Thái Cổ Thần Vương - 3D Siêu Mượt - Tự Do Bay Lượn,1.414214,"Thái Cổ Thần Vương là một game nhập vai 3D mobile với đồ họa và gameplay đẳng cấp PC - dung lượng chỉ 500mb. Đưa bạn vào một thế giới tiên hiệp hoàn mỹ - nhìn đã mắt - chơi sướng tay – mượt mà thân thiện với mọi dòng máy.\nĐến với thế giới Thái Cổ Thần Vương VTC bạn có thể\n\n1. Tự do bay – lượn trong một thế giới bao la rộng lớn\n2. Mãn nhãn với cảnh quan, công trình kiến trúc đồ sộ, tạo hình nhân vật đặc sắc\n3. Được chuyển thể từ truyện và phim cùng tên nên game sở hữu một cốt truyện lay ..."
475,Düşen! - Kelime Oyunu,1.414214,"En güzel yeni tarz Türkçe Kelime / Sözcük Bulmaca Oyunu çok sevildi!\nDüşen Kelime Oyunu - Yeni Tarz Sözcük Bulmaca Oyunu olarak kısa sürede 600 Bin den fazla indirme aldı !\nBulmaca, Kelime Arama Oyunu ve Kelime Bulmaca Oyunu severler başta olmak üzere daha önce hiç oyun oynamamışlar bile bu güzel oyuna bayıldı ve bağımlısı oldu :)\n\n- Ücretsiz ve İnternetsiz gizli kelime bulmaca oyunu\n- Tek elle ve hatta tek parmakla oynanabilen pratik ve eğlenceli bir kelime oyunu\n- Daha önce oynadığın..."
1610,기적의 검,1.414214,"소지섭, 안젤리나가 선택한 바로 그 게임!\n\n★ 공식카페에서 구글 사전예약 한정 특별 선물 배포중! ★\n\n\n이 세상 편함이 아니다!\n세로의 치명적인 매력\n세로운 한손 액션 판타지 rpg 기적의 검\n\n\n1. 이보다 편한 RPG는 없다!\n- 한손 원터치 폭풍 성장과 \n끊임없는 오프

In [26]:
# removing tokens containing non-alphabetical characters
class AppSearcher:
    def fit(self, df):
        """Instantiate and fit all the classes required for the search engine (cf. Problems 1 and 2)."""
        self.df = df
        # YOUR CODE HERE
        # https://stackoverflow.com/questions/41988915/avoiding-loading-spacy-data-in-each-subprocess-when-multiprocessing
        self.nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'textcat'])
        # https://www.davidsbatista.net/blog/2018/02/28/TfidfVectorizer/
        self.vectorizer = TfidfVectorizer(tokenizer=self.preprocess, preprocessor=lambda x: x, token_pattern=None)
        self.X = self.vectorizer.fit_transform(df['description'])
        self.neighbors_nearest = NearestNeighbors(n_neighbors=10)
        self.neighbors_nearest.fit(self.X)
        # raise NotImplementedError()

    def preprocess(self, text):
        """Preprocess the given text (cf. Problem 3)."""
        # YOUR CODE HERE
        # self.nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'textcat'])
        doc = self.nlp(text)
        rm_non_alph = [token.text for token in doc if token.is_alpha==True]
        return(rm_non_alph)
        # raise NotImplementedError()

    def search(self, query):
        """Find the nearest neighbors in `df` for a query string (cf. Problem 2)."""
        # YOUR CODE HERE
        query_vector = self.vectorizer.transform([query])
        # https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html
        neigh_dist, neigh_ind = self.neighbors_nearest.kneighbors(query_vector)
        name = self.df.iloc[neigh_ind[0]]['name']
        description = self.df.iloc[neigh_ind[0]]['description']
        distance = neigh_dist[0]
        return pd.DataFrame({'name': name, 'distance': distance, "description" : description})
        # raise NotImplementedError()

In [27]:
apps = AppSearcher()
apps.fit(df)
apps.search("pile up pancakes")

,name,distance,description
1032,Pancake Tower,0.579292,Let's see how many pancakes you can pile up!!
326,Cooking School: Games for Girls,1.288873,"Children like to help their parents. They especially like to help with cooking . When there is a cooking in the kitchen, it is no way to play. But cooking is a complicated process and often it ends up with a huge mess in the kitchen. But what if you are so eager to cook pancakes, cake or cupcakes? How to cook all that without doing a cleaning after? We have a solution! Home Cooking School with our curious Hippo has opened especially for parents and children! We do not only cook food here. We..."
1235,Solitaire,1.364693,"Solitaire Free by Solitaire Card Games is the #1 klondike solitaire games on android. The solitaire Free is popular and classic card games you know and love.\n\nWe carefully designed a fresh solitaire free modern look, woven into the wonderful solitaire classic feel that everyone loves. \n\nExperience the crisp, clear, and easy to read cards, simple and quick animations, and subtle sounds, in either landscape or portrait views. \n\nYou can move cards with a single tap or drag them to their d..."
656,"Hell’s Cooking — crazy chef burger, kitchen fever",1.369729,"⭐ ⭐ ⭐ ⭐ ⭐ New world of crazy cooking is here. Feel what it means to be a master chef who prepares fantastic fast food in a prominent king kitchen! If you haven't ever tried yourself as a hamburger chef cook, it's possibly the best time for making diner. Download and launch Hell's Cooking — crazy chef burger, kitchen fever HD game and get prepared to jump into a fever and adventurous perfect world of burgers.\n\nNew girls game Hell's Cooking gives you lots of opportunities for your crazy cafe..."
1164,Rummy - Free,1.371071,"Play the famous Rummy card game on your Android Smartphone or Tablet !! \n\nPlay rummy with 2, 3, or 4 players against simulated opponents playing with high-level artificial intelligence. \nThere are a number of rules that can be modified, making this game very faithful to the original. \n\n*** MANY VARIATIONS INCLUDED *** \n\nMany rummy variations are included in the application: \n\n- From 2 to 4 players. \n- Choose the AI level of opponents. \n- Number of cards dealt to each player (from ..."
436,Dr. Panda's Ice Cream Truck,1.377365,"Chocolate? Vanilla? Strawberry? All three!? You decide! In Dr. Panda’s Ice Cream Truck you can mix up all sorts of different flavors with cookies, chocolate, nuts and more to make the perfect ice cream—hundreds of combinations in all.\n\nScoop it!\nThese animals love ice cream, and will eat as much (or little) as you want to serve them. You can make scoops big or small and pile them as high as you want—using any of the ice cream you’ve created!\n\nToppings galore!\nUse chocolate syrup, cooki..."
1442,Turbo Dismount™,1.380881,"The legendary crash simulator is now on Google Play!\n\nPerform death-defying motor stunts, crash into walls, create traffic pile-ups of epic scale - and share the fun!\n\nTurbo Dismount™ is a kinetic tragedy about Mr. Dismount and the cars who love him. It is the official sequel to the wildly popular and immensely successful personal impact simulator - Stair Dismount™. \n\nFEATURES:\n* Flinch-inducing crash physics\n* Crunchy sound effects\n* Delicious slow-mo replay system\n* Multiple vehi..."
1326,TO-FU Oh!SUSHI,1.383255,"You are the veritable sushi master! Prepare your own fun sushi with “Daizu” the skunk!\n\nThis app is designed to allow children to be creative by decorating their original sushi.\n\nServe your delicious, mysterious or impossible sushi to the people of “Tofu Island”! \n\nHow about creating sushi that is totally original and serve it to your beloved guests? Spice it up with tons of wasabi or even sprinkle chocolate and gummy bears for those sweet lovers.\nFeel free to make any kind of sushi y..."
427,Dr. Panda Ice Cream Truck Free,1.384113,"Dr. Panda Ice Cream Truck is FREE for you to play!\n\nChocolate? Vanilla? Straw

In [28]:
apps.search("pancake piling")

,name,distance,description
1298,Stylist Girl - Make Me Gorgeous!,1.375352,"If you’re into makeover games, this one will knock your socks off. And if you’re not into makeover games, this one will change that! \n\nYou’re the hottest new fashion stylist girl in Hollywood, and your awesome reputation is already bringing you fame and fortune! Dress up famous fashion models and VIP celebrities. Bring out their star style and give them beautiful makeovers! Create tons of incredible looks for your clients - find them the perfect outfits for their big events - they’ll love ..."
1167,RusDate - Dating. Знакомства и Общение на Русском,1.414214,"Скачай бесплатно лучший чат знакомств на русском языке RusDate и общайся😉. Найти парня или девушку в активном поиске рядом с тобой, или новых друзей - запросто: взаимный лайк на анкету и вы строите отношения вдвоем 💕. Заметил, что рядом симпатичная незнакомка - действуй🎯. Открывай программу, выбирай «Ищу женщину», ставь пометку «рядом со мной» и смотри анкеты девушек, которые поблизости с точностью до метра. Незнакомка может превратиться в твою девушку и жену👰, незнакомец - в парня и мужа🤵. ..."
1612,컴투스프로야구2019,1.414214,★ 레전드 등급 추가로 더욱 강력해진 2019 메이저 업데이트 ★\n\n# 대한민국 레전드 야구게임! 컴투스프로야구2019 메이저 업데이트!\n- 모바일 야구 최강! 극강의 리얼리티를 자랑하는 컴프야 시리즈를 2019년에도 만나보세요!\n\n# 전설들의 귀환! 레전드 카드 출시\n- 무등산 폭격기 선동열! 철완 최동원! 적토마 이병규! 두목곰 김동주 등 KBO 최고의 선수들의 귀환\n- 레전드 카드를 통해 더욱 강력해진 선수들을 만나보세요!\n\n# 류현진이 등장하는 유일한 KBO 야구게임 컴프야!\n- 코리안 몬스터 류현진 선수를 게임에서 만나보세요.\n\n# 강한자가 살아남는다! 실시간 홈런 대전 홈런로얄 BETA 오픈\n- 하나의 타구가 승부를 좌우하는 실시간 홈런모드 신규 오픈\n- 홈런로얄 모드에서 홈런왕에 도전하세요!\n\n# 판타지 라인업 모드 추가!\n- 예측 야구의 끝판왕! \n- 판타지 라인업에서 실제 선수들의 기록을 예상하여 예측왕에 도전하세요!\n- 판타...
86,Au Mobile VTC – Game nhảy Audition,1.414214,"Bí mật tuyệt vời được hé lộ, những em bé chào đời trong sự mong chờ của hàng triệu cặp đôi.\n\nAU MOBILE THẾ GIỚI ONLINE THU NHỎ\nVới 7 triệu dancer đẹp trai xinh gái cùng cộng đồng lớn mạnh với các hoạt động sôi nổi. Thanh xuân của bạn trở thành khoảng thời gian đáng nhớ nhất, vừa đẹp đẽ vừa đơn thuần, bạn bè luôn ở bên cạnh còn người thương luôn ở ngay trước mắt.\n\nNHẢY THEO CẢM XÚC- HẠNH PHÚC TỪ CON TIM\nĐiệu nhảy đẹp nhất là khi hai trái tim hòa cùng một nhịp, nơi bạn cháy hết mình với ..."
686,Hoşkin HD - İnternetsiz,1.414214,"Hoşkin HD, batak ve king oyununa benzeyen bir kart oyunudur.\n\nÖzellikler\n-3 kişilik oyun modu\n-Internet bağlantısına gerek olmadan istediğiniz zaman oynama\n-HD grafikler\n\nHoşkin nasıl oynanır?\n\nKART DAĞITIMI\nDağıtım saat yönünün tersinedir. Dağıtan kişi 3 oyuncuya 25'er kart dağıtır ve masanın ortasına, kalan 5 kartı yerleştirir.\n\nİHALE SEÇİMİ\n- Koz seçilebilmesi için ihale yapılır. Hoşkin'de Batak 'tan farklı olarak kartların puanları vardır. \n- Bu puanlama doğrultusunu oyun ö..."
1601,من سيربح المليون الموسوعة,1.414214,برنامج المسابقات الشهير (من سيربح المليون) بالنسخة الأصلية للبرنامج ...\nالإصدار الذهبي الموسوعة ميزاته كثيرة نذكر منها :\n\n1- إضافة أكثر من 19 آلف سؤال ليتحول البرنامج من لعبة إلى موسوعة حقيقية\n2- اعتماد نظام الأسئلة المتدرجة في الصعوبة ليصبح الإصدار الذهبي نسخة عن البرنامج التلفزيوني الشهير الذي يحمل نفس الاسم (من سيربح المليون)\n3- إضافة وسيلة المساعدة الجديدة (إمكانية تغيير السؤال بعد السؤال الخامس) \n4- إضافة وسيلة مساعدة أخرى وجديدة (بعد الوصول للسؤال السابع) وهي طلب مساعدة احد أصدقا...
1152,Rising Super Chef - Craze Restaurant Cooking Games,1.414214,"Dive into the master chef’s fever of cooking, feel the cooking madness of serving feverish customers, from town to town, city to city, bakery to cafe, dash for cooking tasty burger & pizza in a crazy kitchen, baking cakes & desserts in the bakery, making all kinds coffee in the cafe, and serving tasty fast food all over the world in the fast-paced Rising Super Chef - Craze Restaurant Cooking Games - a crazy time management cooking game with more than 1000 levels, simply the most unmissable f..."
1212,Skat,1.414214,Spiele Skat wann und wo Du willst!\nStarke Gegner. Erstklassiges Design.\n\nSpiele jederzeit gegen starke Computergegn

In [29]:
# lemmatization + lowercasing all characters
class AppSearcher:
    def fit(self, df):
        """Instantiate and fit all the classes required for the search engine (cf. Problems 1 and 2)."""
        self.df = df
        # YOUR CODE HERE
        # https://stackoverflow.com/questions/41988915/avoiding-loading-spacy-data-in-each-subprocess-when-multiprocessing
        self.nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'textcat'])
        # https://www.davidsbatista.net/blog/2018/02/28/TfidfVectorizer/
        self.vectorizer = TfidfVectorizer(tokenizer=self.preprocess, preprocessor=lambda x: x, token_pattern=None)
        self.X = self.vectorizer.fit_transform(df['description'])
        self.neighbors_nearest = NearestNeighbors(n_neighbors=10)
        self.neighbors_nearest.fit(self.X)
        # raise NotImplementedError()

    def preprocess(self, text):
        """Preprocess the given text (cf. Problem 3)."""
        # YOUR CODE HERE
        # self.nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'textcat'])
        doc = self.nlp(text)
        lemmas = [token.lemma_.lower() for token in doc]
        return(lemmas)
        # raise NotImplementedError()

    def search(self, query):
        """Find the nearest neighbors in `df` for a query string (cf. Problem 2)."""
        # YOUR CODE HERE
        query_vector = self.vectorizer.transform([query])
        # https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html
        neigh_dist, neigh_ind = self.neighbors_nearest.kneighbors(query_vector)
        name = self.df.iloc[neigh_ind[0]]['name']
        description = self.df.iloc[neigh_ind[0]]['description']
        distance = neigh_dist[0]
        return pd.DataFrame({'name': name, 'distance': distance, "description" : description})
        # raise NotImplementedError()

In [30]:
apps = AppSearcher()
apps.fit(df)
apps.search("pile up pancakes")

,name,distance,description
1032,Pancake Tower,0.632462,Let's see how many pancakes you can pile up!!
326,Cooking School: Games for Girls,1.292188,"Children like to help their parents. They especially like to help with cooking . When there is a cooking in the kitchen, it is no way to play. But cooking is a complicated process and often it ends up with a huge mess in the kitchen. But what if you are so eager to cook pancakes, cake or cupcakes? How to cook all that without doing a cleaning after? We have a solution! Home Cooking School with our curious Hippo has opened especially for parents and children! We do not only cook food here. We..."
1235,Solitaire,1.352310,"Solitaire Free by Solitaire Card Games is the #1 klondike solitaire games on android. The solitaire Free is popular and classic card games you know and love.\n\nWe carefully designed a fresh solitaire free modern look, woven into the wonderful solitaire classic feel that everyone loves. \n\nExperience the crisp, clear, and easy to read cards, simple and quick animations, and subtle sounds, in either landscape or portrait views. \n\nYou can move cards with a single tap or drag them to their d..."
1181,Sago Mini Trucks and Diggers,1.353493,"Drive a dump truck with Rosie the hamster! Pile dirt high and dig deep in the ground with diggers, cranes and bulldozers. Build a home for a new friend! Choose a barn, a castle or even a cupcake-house. Don’t forget to add the finishing touches for the proud owner.\n\nOn this construction site, kids love being the boss. With six mighty machines and piles of dirt, you can build all day! Part of the award-winning suite of Sago Mini apps, this app puts kids in charge.\n\nSago Mini apps have no i..."
1263,Spider Solitaire,1.354415,"Spider Solitaire was built to offer card players a fun way to play their favorite classic in both portrait and landscape mode.\n\nWith large cards and a unique stacking system our Spider card game doesn't have problems fitting your screen like many others do. \n\n* How to play *\n\nTo win a game of spider solitaire, all cards must be removed from the table. Assembling the cards in the tableau allows for cards to be placed in their respective stacks in order. At the beginning of each game, 54..."
656,"Hell’s Cooking — crazy chef burger, kitchen fever",1.373275,"⭐ ⭐ ⭐ ⭐ ⭐ New world of crazy cooking is here. Feel what it means to be a master chef who prepares fantastic fast food in a prominent king kitchen! If you haven't ever tried yourself as a hamburger chef cook, it's possibly the best time for making diner. Download and launch Hell's Cooking — crazy chef burger, kitchen fever HD game and get prepared to jump into a fever and adventurous perfect world of burgers.\n\nNew girls game Hell's Cooking gives you lots of opportunities for your crazy cafe..."
1164,Rummy - Free,1.381410,"Play the famous Rummy card game on your Android Smartphone or Tablet !! \n\nPlay rummy with 2, 3, or 4 players against simulated opponents playing with high-level artificial intelligence. \nThere are a number of rules that can be modified, making this game very faithful to the original. \n\n*** MANY VARIATIONS INCLUDED *** \n\nMany rummy variations are included in the application: \n\n- From 2 to 4 players. \n- Choose the AI level of opponents. \n- Number of cards dealt to each player (from ..."
1442,Turbo Dismount™,1.383775,"The legendary crash simulator is now on Google Play!\n\nPerform death-defying motor stunts, crash into walls, create traffic pile-ups of epic scale - and share the fun!\n\nTurbo Dismount™ is a kinetic tragedy about Mr. Dismount and the cars who love him. It is the official sequel to the wildly popular and immensely successful personal impact simulator - Stair Dismount™. \n\nFEATURES:\n* Flinch-inducing crash physics\n* Crunchy sound effects\n* Delicious slow-mo replay system\n* Multiple vehi..."
436,Dr. Panda's Ice Cream Truck,1.384078,"Chocolate? Vanilla? Strawberry? All three!? You decide! In Dr. Panda’s Ice Cre

In [31]:
apps.search("pancake piling")

,name,distance,description
1032,Pancake Tower,0.872878,Let's see how many pancakes you can pile up!!
326,Cooking School: Games for Girls,1.257918,"Children like to help their parents. They especially like to help with cooking . When there is a cooking in the kitchen, it is no way to play. But cooking is a complicated process and often it ends up with a huge mess in the kitchen. But what if you are so eager to cook pancakes, cake or cupcakes? How to cook all that without doing a cleaning after? We have a solution! Home Cooking School with our curious Hippo has opened especially for parents and children! We do not only cook food here. We..."
656,"Hell’s Cooking — crazy chef burger, kitchen fever",1.361076,"⭐ ⭐ ⭐ ⭐ ⭐ New world of crazy cooking is here. Feel what it means to be a master chef who prepares fantastic fast food in a prominent king kitchen! If you haven't ever tried yourself as a hamburger chef cook, it's possibly the best time for making diner. Download and launch Hell's Cooking — crazy chef burger, kitchen fever HD game and get prepared to jump into a fever and adventurous perfect world of burgers.\n\nNew girls game Hell's Cooking gives you lots of opportunities for your crazy cafe..."
1610,기적의 검,1.414214,"소지섭, 안젤리나가 선택한 바로 그 게임!\n\n★ 공식카페에서 구글 사전예약 한정 특별 선물 배포중! ★\n\n\n이 세상 편함이 아니다!\n세로의 치명적인 매력\n세로운 한손 액션 판타지 rpg 기적의 검\n\n\n1. 이보다 편한 RPG는 없다!\n- 한손 원터치 폭풍 성장과 \n끊임없는 오프라인 성장으로 시간을 지배하라!\n\n2. 연맹 영지 탈환을 위한 핏빛 전쟁!\n- 각 영주 자리를 향한 대규모 연맹의 아찔한 쟁패가 시작된다!\n\n3. 여러가지맛 성장 시스템\n- 다양한 개성을 녹여 지루할 틈이 없는 \n성령 / 신병 / 마법진 / 신수 등 다채로운 육성법으로\n무한한 성장의 기쁨을 느껴보세요!\n\n4. 너와 나의 만남은 우연이 아니다!\n- 한 손으로 맺는 아름다운 인연, \n또 하나의 소중한 인연이 당신만을 기다린다.\n\n5. 모으고 꾸미고 성장하라!\n- 화려하고 개성넘치는 다양한 코스튬의 파도속에서\n나만의 개성을 찾아 거침없이 개방하라!\n\n▶기적..."
475,Düşen! - Kelime Oyunu,1.414214,"En güzel yeni tarz Türkçe Kelime / Sözcük Bulmaca Oyunu çok sevildi!\nDüşen Kelime Oyunu - Yeni Tarz Sözcük Bulmaca Oyunu olarak kısa sürede 600 Bin den fazla indirme aldı !\nBulmaca, Kelime Arama Oyunu ve Kelime Bulmaca Oyunu severler başta olmak üzere daha önce hiç oyun oynamamışlar bile bu güzel oyuna bayıldı ve bağımlısı oldu :)\n\n- Ücretsiz ve İnternetsiz gizli kelime bulmaca oyunu\n- Tek elle ve hatta tek parmakla oynanabilen pratik ve eğlenceli bir kelime oyunu\n- Daha önce oynadığın..."
1336,Tallowmere,1.414214,"Raise your shield. Embrace Lady Tallowmere's ever-changing dungeons. How far can you make it?\n\nFeatures:\n• Roguelike platformer\n• Randomly-generated rooms, each bigger than the last\n• Seven tiers of loot, ranging from Basic to Otherworldly\n• Single-player mode (use touchscreen, controller, or keyboard)\n• Local high-score and achievement system\n• Sacrificial kittens\n\nTo those who will be lured into Lady Tallowmere's dungeons: good luck.\n\nPlease note: Tallowmere features blood and ..."
1609,고스톱 PLUS (무료 맞고 게임),1.414214,심심할 때 한번씩 즐기는 고스톱(맞고).\n복잡하고 요란한 화면이 부담스러우셨나요?\n게임이 뚝뚝 끊기는 바람에 답답하셨나요?\n\n그냥 가볍게 고스톱(맞고) 게임을 즐기고 싶은데\n게임머니를 구매해야 해서 불편하셨나요?\n\n그렇다면 '고스톱 PLUS' 를 플레이 해보세요!\n\n'고스톱 PLUS'\n아무 때나 꺼내서 가볍게 한판!\n매끄럽고 깔끔하게-\n구매 걱정없이 무료로-\n고스톱(맞고) 플레이를 즐겨보세요!\n\n※특징\n- 빠른 게임 시작. 매끄러운 고스톱(맞고) 플레이\n- 화투패를 내리치는 생생한 느낌\n- 직접 치기 귀찮을 땐 자동치기\n- 대박판의 긴장감\n- 정말 빠른 고스톱(맞고) 속도\n- 깔끔하고 재미난 애니메이션 효과\n- 눈이 피로하지 않은 깨끗한 디자인\n- 나만의 테마 설정\n- 육성으로 듣기 설정\n- 다양한 효과음 설정\n- 세밀한 규칙 설정\n- 방대한 수준의 레벨 999+α\n- 실시간 기록 경쟁\n- 다양한 업적 달성\n- 상세 통계 ...
1608,機動戦隊アイアンサーガ,1.414214,「ゲーム概要」 \n自動戦闘AIとパーティー編成がカギを握るオリジナル戦闘システム！ \nハイクオリティなドット絵によるバトルメカ、戦闘機、戦車が総勢300体以上用意され、魅力的なキャラクターも多数登場する！\nドラッグ操作で移動+タップでスキルが発動し、リアルな弾道表現とビームやバリアなどのSF要素も満載！ 軽快な操作感と迫力のあるエフェクトによる唯一無二の二次元ロボットバトルを今まさに体験せよ！\n\n【バトルシステム】 \n緊迫感満載の戦闘はリアルタイムで進行！ドラッグ操作で機体を操り、有利なポジションをキープせよ！\n各バトルユニットは、自動戦闘AIにより独自に行動、アクションが苦手な方でも安心してプレイが可能に！ \nステージごとに必要な戦略は様々！どのように戦場を攻略するかはあなたの腕次第。\n\n【パイロット】 \nパイロットたちは酒場にランダムで登場する。彼らに食事をご馳走したり、お酒を奢ったりすることで親密度を高め、好感度を一定値以上に上げて仲間にしよう！ \n早く仲間にするほど育成も早く進む！お気に入りの「パイロット」に遭遇したら決して見逃すな！\n\n【多...
1595,Филворды: поиск слов,1.414214,"Игра Филворды - это несколько сотен уровней интеллектуального наслаждения. Вы ищете загаданные слова, а

In [32]:
# lemmatization + lowercasing all characters + removing stop words
class AppSearcher:
    def fit(self, df):
        """Instantiate and fit all the classes required for the search engine (cf. Problems 1 and 2)."""
        self.df = df
        # YOUR CODE HERE
        # https://stackoverflow.com/questions/41988915/avoiding-loading-spacy-data-in-each-subprocess-when-multiprocessing
        self.nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'textcat'])
        # https://www.davidsbatista.net/blog/2018/02/28/TfidfVectorizer/
        self.vectorizer = TfidfVectorizer(tokenizer=self.preprocess, preprocessor=lambda x: x, token_pattern=None)
        self.X = self.vectorizer.fit_transform(df['description'])
        self.neighbors_nearest = NearestNeighbors(n_neighbors=10)
        self.neighbors_nearest.fit(self.X)
        # raise NotImplementedError()

    def preprocess(self, text):
        """Preprocess the given text (cf. Problem 3)."""
        # YOUR CODE HERE
        # self.nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'textcat'])
        doc = self.nlp(text)
        lemmas = [token.lemma_.lower() for token in doc if token.is_stop == False]
        return(lemmas)
        # raise NotImplementedError()

    def search(self, query):
        """Find the nearest neighbors in `df` for a query string (cf. Problem 2)."""
        # YOUR CODE HERE
        query_vector = self.vectorizer.transform([query])
        # https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html
        neigh_dist, neigh_ind = self.neighbors_nearest.kneighbors(query_vector)
        name = self.df.iloc[neigh_ind[0]]['name']
        description = self.df.iloc[neigh_ind[0]]['description']
        distance = neigh_dist[0]
        return pd.DataFrame({'name': name, 'distance': distance, "description" : description})
        # raise NotImplementedError()

In [33]:
apps = AppSearcher()
apps.fit(df)
apps.search("pile up pancakes")

,name,distance,description
1032,Pancake Tower,0.377223,Let's see how many pancakes you can pile up!!
326,Cooking School: Games for Girls,1.275503,"Children like to help their parents. They especially like to help with cooking . When there is a cooking in the kitchen, it is no way to play. But cooking is a complicated process and often it ends up with a huge mess in the kitchen. But what if you are so eager to cook pancakes, cake or cupcakes? How to cook all that without doing a cleaning after? We have a solution! Home Cooking School with our curious Hippo has opened especially for parents and children! We do not only cook food here. We..."
1181,Sago Mini Trucks and Diggers,1.346693,"Drive a dump truck with Rosie the hamster! Pile dirt high and dig deep in the ground with diggers, cranes and bulldozers. Build a home for a new friend! Choose a barn, a castle or even a cupcake-house. Don’t forget to add the finishing touches for the proud owner.\n\nOn this construction site, kids love being the boss. With six mighty machines and piles of dirt, you can build all day! Part of the award-winning suite of Sago Mini apps, this app puts kids in charge.\n\nSago Mini apps have no i..."
1263,Spider Solitaire,1.348894,"Spider Solitaire was built to offer card players a fun way to play their favorite classic in both portrait and landscape mode.\n\nWith large cards and a unique stacking system our Spider card game doesn't have problems fitting your screen like many others do. \n\n* How to play *\n\nTo win a game of spider solitaire, all cards must be removed from the table. Assembling the cards in the tableau allows for cards to be placed in their respective stacks in order. At the beginning of each game, 54..."
1235,Solitaire,1.351876,"Solitaire Free by Solitaire Card Games is the #1 klondike solitaire games on android. The solitaire Free is popular and classic card games you know and love.\n\nWe carefully designed a fresh solitaire free modern look, woven into the wonderful solitaire classic feel that everyone loves. \n\nExperience the crisp, clear, and easy to read cards, simple and quick animations, and subtle sounds, in either landscape or portrait views. \n\nYou can move cards with a single tap or drag them to their d..."
656,"Hell’s Cooking — crazy chef burger, kitchen fever",1.368401,"⭐ ⭐ ⭐ ⭐ ⭐ New world of crazy cooking is here. Feel what it means to be a master chef who prepares fantastic fast food in a prominent king kitchen! If you haven't ever tried yourself as a hamburger chef cook, it's possibly the best time for making diner. Download and launch Hell's Cooking — crazy chef burger, kitchen fever HD game and get prepared to jump into a fever and adventurous perfect world of burgers.\n\nNew girls game Hell's Cooking gives you lots of opportunities for your crazy cafe..."
1164,Rummy - Free,1.382413,"Play the famous Rummy card game on your Android Smartphone or Tablet !! \n\nPlay rummy with 2, 3, or 4 players against simulated opponents playing with high-level artificial intelligence. \nThere are a number of rules that can be modified, making this game very faithful to the original. \n\n*** MANY VARIATIONS INCLUDED *** \n\nMany rummy variations are included in the application: \n\n- From 2 to 4 players. \n- Choose the AI level of opponents. \n- Number of cards dealt to each player (from ..."
436,Dr. Panda's Ice Cream Truck,1.383313,"Chocolate? Vanilla? Strawberry? All three!? You decide! In Dr. Panda’s Ice Cream Truck you can mix up all sorts of different flavors with cookies, chocolate, nuts and more to make the perfect ice cream—hundreds of combinations in all.\n\nScoop it!\nThese animals love ice cream, and will eat as much (or little) as you want to serve them. You can make scoops big or small and pile them as high as you want—using any of the ice cream you’ve created!\n\nToppings galore!\nUse chocolate syrup, cooki..."
1442,Turbo Dismount™,1.385583,"The legendary crash simulator is now on Google Play!\n\nPerform death-defying 

In [34]:
apps.search("pancake piling")

,name,distance,description
1032,Pancake Tower,0.730324,Let's see how many pancakes you can pile up!!
326,Cooking School: Games for Girls,1.235889,"Children like to help their parents. They especially like to help with cooking . When there is a cooking in the kitchen, it is no way to play. But cooking is a complicated process and often it ends up with a huge mess in the kitchen. But what if you are so eager to cook pancakes, cake or cupcakes? How to cook all that without doing a cleaning after? We have a solution! Home Cooking School with our curious Hippo has opened especially for parents and children! We do not only cook food here. We..."
656,"Hell’s Cooking — crazy chef burger, kitchen fever",1.355923,"⭐ ⭐ ⭐ ⭐ ⭐ New world of crazy cooking is here. Feel what it means to be a master chef who prepares fantastic fast food in a prominent king kitchen! If you haven't ever tried yourself as a hamburger chef cook, it's possibly the best time for making diner. Download and launch Hell's Cooking — crazy chef burger, kitchen fever HD game and get prepared to jump into a fever and adventurous perfect world of burgers.\n\nNew girls game Hell's Cooking gives you lots of opportunities for your crazy cafe..."
742,Kelimelik,1.414214,"Türkiye'nin en çok oynanan kelime oyunu 11 milyon üyeyi geçti! Bu başarıya ulaşmamızı sağlayan tüm oyuncularımıza teşekkür ediyoruz. Siz de Kelimelik ailesine katılmak ve arkadaşlarınızla Türkçe kelime oyunu keyfini yaşamak için hemen ücretsiz oyunumuzu indirin.\n\nOyunumuzun Özellikleri:\n\n★ Kelimelik her yerde! İster mobil cihazdan isterseniz de bilgisayarınızdan oynayabilirsiniz.\n\n★ Aynı anda 20'ye kadar oyun açabilir, bu sayede bekleme olmaksızın oyuna devam edebilirsiniz.\n\n★ Facebo..."
1600,كلمات وصور - كلمة السر الذكية 3,1.414214,نقدم لكم العاب بريك لعبة الذكاء الجديدة والشيقة كلمات وصور بريك وصلة لعبة باسلوب فريد وممتع تناسب جميع افراد العائلة وتقضي معها اجمل الاوقات \nهذه اللعبة جديدة كلياً مبنية على فكرة الكلمات المتقاطعة وكلمة السر ولكن بصورة عصرية ومسلية حيث قمنا بوضع الغاز في كل مرحلة وعليك البحث وفك رموز هذه الالغاز للوصل الى حل اللغز .. جربها !!\n\nكلمات وصور - كلمة السر الذكية 3 مبنية على لعبة كلمة السر ولكن هنا سوف تقوم بالبحث عن معاني الرموز بدلا من البحث عن الكلمات حسب تقييم المركز العالمي للبحوث ان الصور...
1592,Сканворды на русском,1.414214,"""Сканворды на русском""- интереснейшая словесная головоломка, которая не заставит Вас скучать. От создателей Быстрых кроссвордов. Еще больше пересечений слов. Еще проще, веселее и удобнее.\n\nИгра проста в использовании и очень забавна. Решая ""Сканворды на русском"", Вы освежаете свои знания, расширяете словарный запас, тренируете мозг и развиваете интеллект.\n\nСканворды на русском обладают всеми перечисленными признаками: \n• Логическая игра- развлечение, которое стимулирует ум.\n• Головолом..."
1601,من سيربح المليون الموسوعة,1.414214,برنامج المسابقات الشهير (من سيربح المليون) بالنسخة الأصلية للبرنامج ...\nالإصدار الذهبي الموسوعة ميزاته كثيرة نذكر منها :\n\n1- إضافة أكثر من 19 آلف سؤال ليتحول البرنامج من لعبة إلى موسوعة حقيقية\n2- اعتماد نظام الأسئلة المتدرجة في الصعوبة ليصبح الإصدار الذهبي نسخة عن البرنامج التلفزيوني الشهير الذي يحمل نفس الاسم (من سيربح المليون)\n3- إضافة وسيلة المساعدة الجديدة (إمكانية تغيير السؤال بعد السؤال الخامس) \n4- إضافة وسيلة مساعدة أخرى وجديدة (بعد الوصول للسؤال السابع) وهي طلب مساعدة احد أصدقا...
686,Hoşkin HD - İnternetsiz,1.414214,"Hoşkin HD, batak ve king oyununa benzeyen bir kart oyunudur.\n\nÖzellikler\n-3 kişilik oyun modu\n-Internet bağlantısına gerek olmadan istediğiniz zaman oynama\n-HD grafikler\n\nHoşkin nasıl oynanır?\n\nKART DAĞITIMI\nDağıtım saat yönünün tersinedir. Dağıtan kişi 3 oyuncuya 25'er kart dağıtır ve masanın ortasına, kalan 5 kartı yerleştirir.\n\nİHALE SEÇİMİ\n- Koz seçilebilmesi için ihale yapılır. Hoşkin'de Batak 'tan farklı olarak kartların puanları vardır. \n- Bu puanlama doğrultusunu oyun ö..."
844,Lungappen©,1.414214,Med hjälp av Lungappen © kan du lära dig mer om att lyssna på lungor o

In [35]:
# all of them 
class AppSearcher:
    def fit(self, df):
        """Instantiate and fit all the classes required for the search engine (cf. Problems 1 and 2)."""
        self.df = df
        # YOUR CODE HERE
        # https://stackoverflow.com/questions/41988915/avoiding-loading-spacy-data-in-each-subprocess-when-multiprocessing
        self.nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'textcat'])
        # https://www.davidsbatista.net/blog/2018/02/28/TfidfVectorizer/
        self.vectorizer = TfidfVectorizer(tokenizer=self.preprocess, preprocessor=lambda x: x, token_pattern=None)
        self.X = self.vectorizer.fit_transform(df['description'])
        self.neighbors_nearest = NearestNeighbors(n_neighbors=10)
        self.neighbors_nearest.fit(self.X)
        # raise NotImplementedError()

    def preprocess(self, text):
        """Preprocess the given text (cf. Problem 3)."""
        # YOUR CODE HERE
        # self.nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'textcat'])
        doc = self.nlp(text)
        lemmas = [token.lemma_.lower() for token in doc if token.is_stop == False and token.is_alpha==True]
        return(lemmas)
        # raise NotImplementedError()

    def search(self, query):
        """Find the nearest neighbors in `df` for a query string (cf. Problem 2)."""
        # YOUR CODE HERE
        query_vector = self.vectorizer.transform([query])
        # https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html
        neigh_dist, neigh_ind = self.neighbors_nearest.kneighbors(query_vector)
        name = self.df.iloc[neigh_ind[0]]['name']
        description = self.df.iloc[neigh_ind[0]]['description']
        distance = neigh_dist[0]
        return pd.DataFrame({'name': name, 'distance': distance, "description" : description})
        # raise NotImplementedError()

In [36]:
apps = AppSearcher()
apps.fit(df)
apps.search("pile up pancakes")

,name,distance,description
1032,Pancake Tower,0.289759,Let's see how many pancakes you can pile up!!
326,Cooking School: Games for Girls,1.269389,"Children like to help their parents. They especially like to help with cooking . When there is a cooking in the kitchen, it is no way to play. But cooking is a complicated process and often it ends up with a huge mess in the kitchen. But what if you are so eager to cook pancakes, cake or cupcakes? How to cook all that without doing a cleaning after? We have a solution! Home Cooking School with our curious Hippo has opened especially for parents and children! We do not only cook food here. We..."
1181,Sago Mini Trucks and Diggers,1.338611,"Drive a dump truck with Rosie the hamster! Pile dirt high and dig deep in the ground with diggers, cranes and bulldozers. Build a home for a new friend! Choose a barn, a castle or even a cupcake-house. Don’t forget to add the finishing touches for the proud owner.\n\nOn this construction site, kids love being the boss. With six mighty machines and piles of dirt, you can build all day! Part of the award-winning suite of Sago Mini apps, this app puts kids in charge.\n\nSago Mini apps have no i..."
1235,Solitaire,1.340727,"Solitaire Free by Solitaire Card Games is the #1 klondike solitaire games on android. The solitaire Free is popular and classic card games you know and love.\n\nWe carefully designed a fresh solitaire free modern look, woven into the wonderful solitaire classic feel that everyone loves. \n\nExperience the crisp, clear, and easy to read cards, simple and quick animations, and subtle sounds, in either landscape or portrait views. \n\nYou can move cards with a single tap or drag them to their d..."
1263,Spider Solitaire,1.345276,"Spider Solitaire was built to offer card players a fun way to play their favorite classic in both portrait and landscape mode.\n\nWith large cards and a unique stacking system our Spider card game doesn't have problems fitting your screen like many others do. \n\n* How to play *\n\nTo win a game of spider solitaire, all cards must be removed from the table. Assembling the cards in the tableau allows for cards to be placed in their respective stacks in order. At the beginning of each game, 54..."
656,"Hell’s Cooking — crazy chef burger, kitchen fever",1.363660,"⭐ ⭐ ⭐ ⭐ ⭐ New world of crazy cooking is here. Feel what it means to be a master chef who prepares fantastic fast food in a prominent king kitchen! If you haven't ever tried yourself as a hamburger chef cook, it's possibly the best time for making diner. Download and launch Hell's Cooking — crazy chef burger, kitchen fever HD game and get prepared to jump into a fever and adventurous perfect world of burgers.\n\nNew girls game Hell's Cooking gives you lots of opportunities for your crazy cafe..."
1164,Rummy - Free,1.372436,"Play the famous Rummy card game on your Android Smartphone or Tablet !! \n\nPlay rummy with 2, 3, or 4 players against simulated opponents playing with high-level artificial intelligence. \nThere are a number of rules that can be modified, making this game very faithful to the original. \n\n*** MANY VARIATIONS INCLUDED *** \n\nMany rummy variations are included in the application: \n\n- From 2 to 4 players. \n- Choose the AI level of opponents. \n- Number of cards dealt to each player (from ..."
436,Dr. Panda's Ice Cream Truck,1.379078,"Chocolate? Vanilla? Strawberry? All three!? You decide! In Dr. Panda’s Ice Cream Truck you can mix up all sorts of different flavors with cookies, chocolate, nuts and more to make the perfect ice cream—hundreds of combinations in all.\n\nScoop it!\nThese animals love ice cream, and will eat as much (or little) as you want to serve them. You can make scoops big or small and pile them as high as you want—using any of the ice cream you’ve created!\n\nToppings galore!\nUse chocolate syrup, cooki..."
1442,Turbo Dismount™,1.381800,"The legendary crash simulator is now on Google Play!\n\nPerform death-defying 

In [37]:
apps.search("pancake piling")

,name,distance,description
1032,Pancake Tower,0.698081,Let's see how many pancakes you can pile up!!
326,Cooking School: Games for Girls,1.227891,"Children like to help their parents. They especially like to help with cooking . When there is a cooking in the kitchen, it is no way to play. But cooking is a complicated process and often it ends up with a huge mess in the kitchen. But what if you are so eager to cook pancakes, cake or cupcakes? How to cook all that without doing a cleaning after? We have a solution! Home Cooking School with our curious Hippo has opened especially for parents and children! We do not only cook food here. We..."
656,"Hell’s Cooking — crazy chef burger, kitchen fever",1.349861,"⭐ ⭐ ⭐ ⭐ ⭐ New world of crazy cooking is here. Feel what it means to be a master chef who prepares fantastic fast food in a prominent king kitchen! If you haven't ever tried yourself as a hamburger chef cook, it's possibly the best time for making diner. Download and launch Hell's Cooking — crazy chef burger, kitchen fever HD game and get prepared to jump into a fever and adventurous perfect world of burgers.\n\nNew girls game Hell's Cooking gives you lots of opportunities for your crazy cafe..."
1587,Детские песни плеер для малышей,1.414214,"*** Самый большой каталог песен!!! *** Работает без интернета !! *** Бесплатно !\nЕще раз повторю приложение полностью бесплатное, так как делал для своей доченьки.\n\nПохож на детский телефон с песенками, на экране изображены картинки из мультиков, ребенок сам без родителей может понять, какая это песня и включить ее. \n\nВ игре есть функция ускорить песенку, песенки начинают петь тонким голоском, моя дочка смеется, ребенок счастлив - счастливы и родители! Детские песни развивают ребенка, п..."
1612,컴투스프로야구2019,1.414214,★ 레전드 등급 추가로 더욱 강력해진 2019 메이저 업데이트 ★\n\n# 대한민국 레전드 야구게임! 컴투스프로야구2019 메이저 업데이트!\n- 모바일 야구 최강! 극강의 리얼리티를 자랑하는 컴프야 시리즈를 2019년에도 만나보세요!\n\n# 전설들의 귀환! 레전드 카드 출시\n- 무등산 폭격기 선동열! 철완 최동원! 적토마 이병규! 두목곰 김동주 등 KBO 최고의 선수들의 귀환\n- 레전드 카드를 통해 더욱 강력해진 선수들을 만나보세요!\n\n# 류현진이 등장하는 유일한 KBO 야구게임 컴프야!\n- 코리안 몬스터 류현진 선수를 게임에서 만나보세요.\n\n# 강한자가 살아남는다! 실시간 홈런 대전 홈런로얄 BETA 오픈\n- 하나의 타구가 승부를 좌우하는 실시간 홈런모드 신규 오픈\n- 홈런로얄 모드에서 홈런왕에 도전하세요!\n\n# 판타지 라인업 모드 추가!\n- 예측 야구의 끝판왕! \n- 판타지 라인업에서 실제 선수들의 기록을 예상하여 예측왕에 도전하세요!\n- 판타...
1592,Сканворды на русском,1.414214,"""Сканворды на русском""- интереснейшая словесная головоломка, которая не заставит Вас скучать. От создателей Быстрых кроссвордов. Еще больше пересечений слов. Еще проще, веселее и удобнее.\n\nИгра проста в использовании и очень забавна. Решая ""Сканворды на русском"", Вы освежаете свои знания, расширяете словарный запас, тренируете мозг и развиваете интеллект.\n\nСканворды на русском обладают всеми перечисленными признаками: \n• Логическая игра- развлечение, которое стимулирует ум.\n• Головолом..."
1600,كلمات وصور - كلمة السر الذكية 3,1.414214,نقدم لكم العاب بريك لعبة الذكاء الجديدة والشيقة كلمات وصور بريك وصلة لعبة باسلوب فريد وممتع تناسب جميع افراد العائلة وتقضي معها اجمل الاوقات \nهذه اللعبة جديدة كلياً مبنية على فكرة الكلمات المتقاطعة وكلمة السر ولكن بصورة عصرية ومسلية حيث قمنا بوضع الغاز في كل مرحلة وعليك البحث وفك رموز هذه الالغاز للوصل الى حل اللغز .. جربها !!\n\nكلمات وصور - كلمة السر الذكية 3 مبنية على لعبة كلمة السر ولكن هنا سوف تقوم بالبحث عن معاني الرموز بدلا من البحث عن الكلمات حسب تقييم المركز العالمي للبحوث ان الصور...
1593,Слово по подсказке Ассоциации - Логические игры,1.414214,"Популярная игра в ассоциации поможет вам расшевелить мозги при решении головоломок! Игра на логику, вам нужно угадать слово по 5 ассоциациям, поочередно открывая их. Например: по ассоциациям ""Звезда"", ""Вакуум"", ""Вселенная"", ""Гагарин"" и ""Бесконечность"" становится понятно, что загадано слово ""Космос"". Открывай слова поочередно и как только догадаешься, что загадано, пробуй давать ответ. Чем меньше слов будет открыто, тем больше монет ты сможешь заработать. \n\nВам встретятся увлекательные и ин..."
1167,RusDate - Dating. Знакомства и Общение на Р

## Individual reflection

<div class="alert alert-info">
    <strong>After you have solved the lab,</strong> write a <em>brief</em> reflection (max. one A4 page) on the question(s) below.  Remember:
    <ul>
        <li>You are encouraged to discuss this part with your lab partner, but you should each write up your reflection <strong>individually</strong>.</li>
        <li><strong>Do not put your answers in the notebook</strong>; upload them in the separate submission opportunity for the reflections on Lisam.</li>
    </ul>
</div>

1. In Problem 1, which token had the highest tf–idf score, which the lowest?  Based on your knowledge of how tf–idf works, how would you explain this result?
2. Based on your observations in Problem 4, which preprocessing steps do you think are the most appropriate for this "search engine" example?  Why?

**Congratulations on finishing this lab! 👍**

<div class="alert alert-info">
    
➡️ Before you submit, **make sure the notebook can be run from start to finish** without errors.  For this, _restart the kernel_ and _run all cells_ from top to bottom. In Jupyter Notebook version 7 or higher, you can do this via "Run$\rightarrow$Restart Kernel and Run All Cells..." in the menu (or the "⏩" button in the toolbar).

</div>